# <div align="center" style="color: #ff5733;">Weekly KPI Report (Recovery Rate by Bucket Added)</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

C:\Users\DwaipayanChakroborti\AppData\Roaming\Python\Python310\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Put the Date Range

In [2]:
sd = '2024-04-22'
ed = '2024-04-28'

# Welcome call Connect - Weekly

In [3]:
sq = f"""WITH
  dt AS (
  SELECT
    DATE('{sd}') WkstartDate,
    DATE('{ed}') WkendDate),
disb as (select distinct disbursementDateTime, customerId, loanType, loanAccountNumber, SUBSTR(mobileNo, -10) as mobileno , loanPaidStatus
, row_number() over(partition by SUBSTR(mobileNo, -10) order by SUBSTR(mobileNo, -10))rnk
from `risk_credit_mis.loan_master_table`, dt 
where 
date_trunc(disbursementDateTime, day) <= dt.WkendDate
-- between dt.WkstartDate and dt.WkendDate 
and 
coalesce(loanPaidStatus, 'NA') not in ('Settled','Completed')
),
wc1 as 
(
SELECT
  dt.WkstartDate,
  dt.WkendDate,
  Genesys_call_id,
  callDatetime,
  COALESCE(IS_APC, 0) APC,
  COALESCE(IS_RPC, 0) RPC,
  COALESCE(IS_PTP,0) PTP,
  RIGHT(mobileNumber, 10) mobileNumber,
  g.campaignName,
  g.agentGroup,
  g.agentFullName,
  g.employeeId,
  g.connected,
  g.notConnected,
  g.callResult,
  ROW_NUMBER() OVER(PARTITION BY Genesys_call_id ORDER BY callDatetime DESC, COALESCE(IS_APC, 0), COALESCE(IS_RPC, 0), COALESCE(IS_PTP, 0))rnk
FROM
  `risk_credit_mis.call_attempt_history_gensys` g,  dt
LEFT JOIN
  `dap_ds_poweruser_playground.campaignmaster` cm
ON
  UPPER(cm.Campaignname) = UPPER(g.campaignName)
INNER JOIN disb ON  SUBSTR(disb.mobileNo, -10) = SUBSTR(CAST(g.mobileNumber AS string), -10)
WHERE
  cm.reference = 'WC'
  and disb.rnk = 1
  and date(callDatetime) between  dt.WkstartDate and dt.WkendDate
), 
wc as 
(select wc1.WkstartDate, wc1.WkendDate, Genesys_call_id,
callDatetime,
APC,
RPC,
PTP,
case when APC = 1 then mobileNumber end apc_m,
case when RPC = 1 then mobileNumber end rpc_m,
case when PTP = 1 then mobileNumber end ptp_m,
mobileNumber,
campaignName,
agentGroup,
agentFullName,
employeeId,
connected,
notConnected,
callResult,
rnk,
from wc1 
where rnk = 1
)
,
wcbase as 
( 
select wc.WkstartDate, wc.WkendDate,
 Genesys_call_id,
callDatetime,
APC,
RPC,
PTP,
mobileNumber,
campaignName,
agentGroup,
agentFullName,
employeeId,
connected,
notConnected,
callResult,
rnk,
apc_m,
rpc_m,
ptp_m
from wc
)
select
(select WkstartDate from dt) WkstartDate,
(select WkendDate from dt) WkendDate,
(select count(distinct apc_m) from wcbase) apc_wc_connected,
(select count(distinct mobileNumber) wccontactrate from wcbase) wccalled,
"""

df_wcconnect_monthly = client.query(sq).to_dataframe(progress_bar_type='tqdm')
df_wcconnect_monthly['WelcomeCall_ContactRate'] = df_wcconnect_monthly['apc_wc_connected']/df_wcconnect_monthly['wccalled']
df_wcconnect_monthly

Job ID 0b19f3d3-fcb8-48cb-a5d0-0461fe746f0f successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,WkstartDate,WkendDate,apc_wc_connected,wccalled,WelcomeCall_ContactRate
0,2024-04-22,2024-04-28,1925,3061,0.628879


This SQL code seems to be extracting data related to welcome calls from a database, likely in a Google BigQuery environment. Let me explain the logic step by step:

1. **Common Table Expressions (CTE): `dt`**
   - This CTE (Common Table Expression) creates a temporary table (`dt`) with the start and end dates provided (`sd` and `ed`), used for filtering later.

2. **Common Table Expressions (CTE): `disb`**
   - This CTE retrieves distinct records from the `risk_credit_mis.loan_master_table` table, filtering for disbursement events that occurred on or before the provided end date (`WkendDate`).
   - The `rnk` column is calculated using the `ROW_NUMBER()` function partitioned by the last 10 digits of the mobile number.
   - The result is filtered to exclude records with a `loanPaidStatus` of 'Settled' or 'Completed'.

3. **Common Table Expressions (CTE): `wc1`**
   - This CTE retrieves data from the `risk_credit_mis.call_attempt_history_gensys` table related to welcome calls (`cm.reference = 'WC'`).
   - The result is joined with the previously defined `disb` CTE using the last 10 digits of the mobile numbers.
   - The `ROW_NUMBER()` function is used to assign a rank to each record based on `callDatetime` and the flags `IS_APC`, `IS_RPC`, and `IS_PTP`.
   - The result is filtered to include only records with `rnk = 1`.

4. **Common Table Expressions (CTE): `wc`**
   - This CTE further refines the data from `wc1`. It includes only records where `rnk = 1`.

5. **Common Table Expressions (CTE): `wcbase`**
   - This CTE is a copy of the previous `wc` CTE, retaining all columns.

6. **Main Query**
   - The main query selects various aggregated counts and values from the CTEs, providing metrics related to welcome calls.
   - The counts include the number of connected calls (`apc_wc_connected`) and the total number of distinct called mobile numbers (`wccontactrate`).
   - Finally, it calculates the contact rate as the ratio of connected calls to the total called numbers.

Overall, this code is designed to analyze and report metrics related to welcome calls, specifically focusing on connected calls and contact rates over a specified time period.

# Reminder Call Contact Rate

In [4]:
sq = f"""WITH
  dt AS (
  SELECT
    DATE('{sd}') WkstartDate,
    DATE('{ed}') WkendDate),
disb as (select distinct disbursementDateTime, customerId, loanType, loanAccountNumber, SUBSTR(mobileNo, -10) as mobileno , loanPaidStatus
, row_number() over(partition by SUBSTR(mobileNo, -10) order by SUBSTR(mobileNo, -10))rnk
from `risk_credit_mis.loan_master_table`, dt 
where 
date_trunc(disbursementDateTime, day) <= dt.WkendDate
-- between dt.WkstartDate and dt.WkendDate 
and 
coalesce(loanPaidStatus, 'NA') not in ('Settled','Completed')
),
rc1 as 
(
SELECT
  dt.WkstartDate,
  dt.WkendDate,
  Genesys_call_id,
  callDatetime,
  COALESCE(IS_APC, 0) APC,
  COALESCE(IS_RPC, 0) RPC,
  COALESCE(IS_PTP,0) PTP,
  RIGHT(mobileNumber, 10) mobileNumber,
  g.campaignName,
  g.agentGroup,
  g.agentFullName,
  g.employeeId,
  g.connected,
  g.notConnected,
  g.callResult,
  ROW_NUMBER() OVER(PARTITION BY Genesys_call_id ORDER BY callDatetime DESC, COALESCE(IS_APC, 0), COALESCE(IS_RPC, 0), COALESCE(IS_PTP, 0))rnk
FROM
  `risk_credit_mis.call_attempt_history_gensys` g,  dt
LEFT JOIN
  `dap_ds_poweruser_playground.campaignmaster` cm
ON
  UPPER(cm.Campaignname) = UPPER(g.campaignName)
INNER JOIN disb ON  SUBSTR(disb.mobileNo, -10) = SUBSTR(CAST(g.mobileNumber AS string), -10)
WHERE
  cm.reference = 'REM'
  and disb.rnk = 1
  and date(callDatetime) between  dt.WkstartDate and dt.WkendDate
), 
rc as 
(select rc1.WkstartDate, rc1.WkendDate, Genesys_call_id,
callDatetime,
APC,
RPC,
PTP,
case when APC = 1 then mobileNumber end apc_m,
case when RPC = 1 then mobileNumber end rpc_m,
case when PTP = 1 then mobileNumber end ptp_m,
mobileNumber,
campaignName,
agentGroup,
agentFullName,
employeeId,
connected,
notConnected,
callResult,
rnk,
from rc1 
where rnk = 1
)
,
rcbase as 
( 
select rc.WkstartDate, rc.WkendDate,
 Genesys_call_id,
callDatetime,
APC,
RPC,
PTP,
mobileNumber,
campaignName,
agentGroup,
agentFullName,
employeeId,
connected,
notConnected,
callResult,
rnk,
apc_m,
rpc_m,
ptp_m
from rc
)
select
(select WkstartDate from dt) WkstartDate,
(select WkendDate from dt) WkendDate,
(select count(distinct apc_m) from rcbase) apc_rc_connected,
(select count(distinct mobileNumber) rccontactrate from rcbase) rccalled
"""
dfremindercall = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfremindercall['remindercallcontactrate'] = dfremindercall['apc_rc_connected']/dfremindercall['rccalled']
dfremindercall


Job ID 5c86f5a6-6db1-4df2-8dbe-fec65d261bd5 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,WkstartDate,WkendDate,apc_rc_connected,rccalled,remindercallcontactrate
0,2024-04-22,2024-04-28,3314,8464,0.391541


In [5]:
df = pd.concat([df_wcconnect_monthly,dfremindercall[['apc_rc_connected', 'rccalled', 'remindercallcontactrate']]], axis = 1)
df

,WkstartDate,WkendDate,apc_wc_connected,wccalled,WelcomeCall_ContactRate,apc_rc_connected,rccalled,remindercallcontactrate
0,2024-04-22,2024-04-28,1925,3061,0.628879,3314,8464,0.391541


This SQL code appears to be querying data related to reminder calls from a database, potentially in a Google BigQuery environment. Let's break down the logic step by step:

1. **Common Table Expressions (CTE): `dt`**
   - This CTE creates a temporary table (`dt`) with the start and end dates provided (`sd` and `ed`), used for filtering later.

2. **Common Table Expressions (CTE): `disb`**
   - This CTE retrieves distinct records from the `risk_credit_mis.loan_master_table` table, filtering for disbursement events that occurred on or before the provided end date (`WkendDate`).
   - The `rnk` column is calculated using the `ROW_NUMBER()` function partitioned by the last 10 digits of the mobile number.
   - The result is filtered to exclude records with a `loanPaidStatus` of 'Settled' or 'Completed'.

3. **Common Table Expressions (CTE): `rc1`**
   - This CTE retrieves data from the `risk_credit_mis.call_attempt_history_gensys` table related to reminder calls (`cm.reference = 'REM'`).
   - The result is joined with the previously defined `disb` CTE using the last 10 digits of the mobile numbers.
   - The `ROW_NUMBER()` function is used to assign a rank to each record based on `callDatetime` and the flags `IS_APC`, `IS_RPC`, and `IS_PTP`.
   - The result is filtered to include only records with `rnk = 1`.

4. **Common Table Expressions (CTE): `rc`**
   - This CTE further refines the data from `rc1`. It includes only records where `rnk = 1`.

5. **Common Table Expressions (CTE): `rcbase`**
   - This CTE is a copy of the previous `rc` CTE, retaining all columns.

6. **Main Query**
   - The main query selects various aggregated counts and values from the CTEs, providing metrics related to reminder calls.
   - The counts include the number of connected calls (`apc_rc_connected`) and the total number of distinct called mobile numbers (`rccontactrate`).
   - Finally, it calculates the contact rate as the ratio of connected calls to the total called numbers.

Overall, this code is designed to analyze and report metrics related to reminder calls, specifically focusing on connected calls and contact rates over a specified time period. The logic is quite similar to the previous code related to welcome calls, with adjustments made for the specific criteria and metrics related to reminder calls.

# Bucket 1 contact Rate

In [6]:
sq = f"""WITH
  dt AS (
  SELECT
    DATE('{sd}') WkstartDate,
    DATE('{ed}') WkendDate),
disb as (select distinct disbursementDateTime, customerId, loanType, loanAccountNumber, SUBSTR(mobileNo, -10) as mobileno , loanPaidStatus
, row_number() over(partition by SUBSTR(mobileNo, -10) order by SUBSTR(mobileNo, -10))rnk
from `risk_credit_mis.loan_master_table`, dt 
where 
date_trunc(disbursementDateTime, day) <= dt.WkendDate
-- between dt.WkstartDate and dt.WkendDate 
and 
coalesce(loanPaidStatus, 'NA') not in ('Settled','Completed')
),
cd AS (
  SELECT
    customer_id,
    new_mobile_number,
    ROW_NUMBER() OVER(PARTITION BY customer_id ORDER BY change_date)rnk
  FROM
    `risk_credit_mis.customer_contact_details`
  WHERE
    ACTIVE = 'Y')
  ,
  buck1 as 
  (select distinct dt.WkstartDate, dt.WkendDate, lbfrc.loanAccountNumber, lbfrc.loanStatus, lbfrc.bucketDate, lbfrc.Max_current_DPD as DPD, lmt.customerId, right(cd.new_mobile_number, 10) newmobilenumber
  , row_number() over(partition by lbfrc.loanAccountNumber order by lbfrc.bucketDate desc) buck1rnk
  from `risk_credit_mis.loan_bucket_flow_report_core` lbfrc, dt 
  left join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = lbfrc.loanAccountNumber
  left join cd on cd.customer_id = lmt.customerId
  where date(bucketDate) between  date(dt.WkstartDate) and  date(dt.WkendDate)
  and coalesce(lbfrc.Max_current_DPD, 0) between 1 and 30 
  and date(lbfrc.firstDueDate) <= date(current_date())
  and lbfrc.loanStatus = 'In Arrears'
  and cd.rnk = 1
  )
    ,
buck1cr1 as 
(select distinct
 dt.WkstartDate,
  dt.WkendDate,
  g.Genesys_call_id,
  g.callDatetime,
  COALESCE(g.IS_APC, 0) APC,
  COALESCE(g.IS_RPC, 0) RPC,
  COALESCE(g.IS_PTP,0) PTP,
  RIGHT(g.mobileNumber, 10) mobileNumber,
  buck1.newmobilenumber mobilefrombuckettable,
  g.campaignName,
  g.agentGroup,
  g.agentFullName,
  g.employeeId,
  g.connected,
  g.notConnected,
  g.callResult, 
  buck1.loanAccountNumber,
  buck1.DPD,
  case when g.IS_APC = 1 then g.mobileNumber end apc_m,
  case when g.IS_RPC = 1 then g.mobileNumber end rpc_m,
  case when g.IS_PTP = 1 then mobileNumber end ptp_m,
  ROW_NUMBER() OVER(PARTITION BY g.Genesys_call_id ORDER BY g.callDatetime DESC, COALESCE(g.IS_APC, 0), COALESCE(g.IS_RPC, 0), COALESCE(g.IS_PTP, 0))rnk 
from  `risk_credit_mis.call_attempt_history_gensys` g,  dt
inner join buck1 on buck1.newmobilenumber = right(g.mobileNumber,10)
LEFT JOIN  `dap_ds_poweruser_playground.campaignmaster` cm ON  UPPER(cm.Campaignname) = UPPER(g.campaignName)
where cm.Reference = 'COLL' and cm.Subcategory = 'SOFT'
AND DATE(g.callDatetime) BETWEEN DATE(dt.WkstartDate) AND DATE(dt.WkendDate)
and buck1.buck1rnk = 1

)
,
buck1cr1base as 
(select buck1cr1.WkstartDate, buck1cr1.WkendDate, buck1cr1.Genesys_call_id, buck1cr1.callDatetime
, buck1cr1.mobileNumber
, buck1cr1.mobilefrombuckettable
, buck1cr1.campaignName
, buck1cr1.agentGroup
, buck1cr1.agentFullName
, buck1cr1.employeeId
, buck1cr1.connected
, buck1cr1.notConnected
, buck1cr1.callResult
, buck1cr1.loanAccountNumber
, buck1cr1.apc_m 
, buck1cr1.rpc_m
, buck1cr1.ptp_m 
, buck1cr1.DPD
, buck1cr1.rnk 
 from buck1cr1 where rnk = 1
)
Select * 
, (select count(distinct apc_m) from buck1cr1base) bucket1contacted,
(select count(distinct mobilefrombuckettable) Bucket1callcontactrate from buck1cr1base where mobilefrombuckettable is not null)bucketcallcontacted
from dt;"""

dfbucket1contactrate = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfbucket1contactrate['bucket1contactrate'] = dfbucket1contactrate['bucket1contacted']/dfbucket1contactrate['bucketcallcontacted']
dfbucket1contactrate

Job ID 0553a453-562a-40dd-8099-1d09b4db814f successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,WkstartDate,WkendDate,bucket1contacted,bucketcallcontacted,bucket1contactrate
0,2024-04-22,2024-04-28,723,4343,0.166475


In [7]:
df = pd.concat([df,dfbucket1contactrate[['bucket1contacted', 'bucketcallcontacted', 'bucket1contactrate']]], axis = 1)
df

,WkstartDate,WkendDate,apc_wc_connected,wccalled,WelcomeCall_ContactRate,apc_rc_connected,rccalled,remindercallcontactrate,bucket1contacted,bucketcallcontacted,bucket1contactrate
0,2024-04-22,2024-04-28,1925,3061,0.628879,3314,8464,0.391541,723,4343,0.166475


This SQL code appears to be querying data related to a specific type of call campaign, likely associated with loan collections or debt recovery. Let's break down the logic step by step:

1. **Common Table Expressions (CTE): `dt`**
   - Similar to previous examples, this CTE creates a temporary table (`dt`) with the start and end dates provided (`sd` and `ed`), used for filtering later.

2. **Common Table Expressions (CTE): `disb`**
   - This CTE retrieves distinct records from the `risk_credit_mis.loan_master_table` table, filtering for disbursement events that occurred on or before the provided end date (`WkendDate`).
   - The `rnk` column is calculated using the `ROW_NUMBER()` function partitioned by the last 10 digits of the mobile number.
   - The result is filtered to exclude records with a `loanPaidStatus` of 'Settled' or 'Completed'.

3. **Common Table Expressions (CTE): `cd`**
   - This CTE retrieves data from the `risk_credit_mis.customer_contact_details` table, including the customer ID and new mobile number, with row numbers assigned based on the change date.

4. **Common Table Expressions (CTE): `buck1`**
   - This CTE retrieves data related to loans in arrears from the `risk_credit_mis.loan_bucket_flow_report_core` table, considering only loans with a maximum current DPD (Days Past Due) between 1 and 30.
   - It also joins with the previously defined `disb` CTE and applies additional filters.
   - The result is assigned a row number based on the bucket date in descending order.

5. **Common Table Expressions (CTE): `buck1cr1`**
   - This CTE retrieves call attempt data related to collections from the `risk_credit_mis.call_attempt_history_gensys` table.
   - It joins with the `buck1` CTE and includes details such as campaign information, call results, and row numbers based on the call datetime and flags for different call types (APC, RPC, PTP).
   - The result is filtered to include only records with `rnk = 1` and calls associated with the 'COLL' reference and 'SOFT' subcategory.

6. **Common Table Expressions (CTE): `buck1cr1base`**
   - This CTE is a copy of the previous `buck1cr1` CTE, retaining all columns.

7. **Main Query**
   - The main query selects various aggregated counts and values from the CTEs, providing metrics related to calls associated with loans in arrears.
   - The counts include the number of connected calls (`bucket1contacted`) and the total number of distinct mobile numbers contacted (`Bucket1callcontactrate`).
   - Finally, it calculates the contact rate as the ratio of connected calls to the total called numbers.

Overall, this code is designed to analyze and report metrics related to calls associated with loans in arrears, specifically focusing on connected calls and contact rates over a specified time period. The logic involves joining and filtering data from multiple tables to extract relevant information for analysis.

# Bucket 1 Normalization (Greg)

In [8]:
sq = f"""WITH
  dt AS (
  SELECT
    DATE('{sd}') WkstartDate,
    DATE('{ed}') WkendDate),
disb as (select distinct disbursementDateTime, customerId, loanType, loanAccountNumber, SUBSTR(mobileNo, -10) as mobileno , loanPaidStatus
, row_number() over(partition by SUBSTR(mobileNo, -10) order by SUBSTR(mobileNo, -10))rnk
from `risk_credit_mis.loan_master_table`, dt 
where 
date_trunc(disbursementDateTime, day) <= dt.WkendDate
-- between dt.WkstartDate and dt.WkendDate 
and 
coalesce(loanPaidStatus, 'NA') not in ('Settled','Completed')
),
cd AS (
  SELECT
    customer_id,
    new_mobile_number,
    ROW_NUMBER() OVER(PARTITION BY customer_id ORDER BY change_date)rnk
  FROM
    `risk_credit_mis.customer_contact_details`
  WHERE
    ACTIVE = 'Y')
  ,
buck1 as 
(select distinct dt.WkstartDate, dt.WkendDate, lbfrc.bucketDate, lbfrc.loanAccountNumber, lbfrc.loanStatus, lbfrc.bucketDate, lbfrc.Max_current_DPD as DPD
  , lmt.customerId, right(cd.new_mobile_number, 10) newmobilenumber from `risk_credit_mis.loan_bucket_flow_report_core` lbfrc, dt 
  left join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = lbfrc.loanAccountNumber
  left join cd on cd.customer_id = lmt.customerId
  where date(bucketDate) =  date(dt.WkstartDate) and coalesce(lbfrc.Max_current_DPD, 0) between 1 and 30 ),
buck1end as
(select distinct dt.WkstartDate, dt.WkendDate, lbfrc.bucketDate, lbfrc.loanAccountNumber, lbfrc.loanStatus, lbfrc.bucketDate, lbfrc.Max_current_DPD as DPD
  , lmt.customerId, right(cd.new_mobile_number, 10) newmobilenumber 
from `risk_credit_mis.loan_bucket_flow_report_core` lbfrc, dt 
inner join buck1 on buck1.loanAccountNumber = lbfrc.loanAccountNumber
left join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = lbfrc.loanAccountNumber
left join cd on cd.customer_id = lmt.customerId
where date(lbfrc.bucketDate) =  date(dt.WkendDate))
select  dt.WkstartDate, dt.WkendDate
, count(distinct case when coalesce(buck1end.DPD, 0) <= coalesce(buck1.DPD, 0) then buck1.loanAccountNumber end) Normalized_Greg
, count(distinct buck1.loanAccountNumber) Normalized_denominator
from buck1, dt
left join buck1end on buck1end.loanAccountNumber = buck1.loanAccountNumber
group by 1, 2

"""
dfbuck1normalizationgreg = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfbuck1normalizationgreg['Bucket1_Normalized'] = dfbuck1normalizationgreg['Normalized_Greg']/dfbuck1normalizationgreg['Normalized_denominator']
dfbuck1normalizationgreg

Job ID 1e035c1e-f7f8-40c3-86ed-f39228f4d944 successfully executed: |          |
Downloading: 100%|██████████|


,WkstartDate,WkendDate,Normalized_Greg,Normalized_denominator,Bucket1_Normalized
0,2024-04-22,2024-04-28,530,3472,0.15265


In [9]:
df = pd.concat([df, dfbuck1normalizationgreg[['Normalized_Greg','Normalized_denominator','Bucket1_Normalized']]], axis = 1)
df

,WkstartDate,WkendDate,apc_wc_connected,wccalled,WelcomeCall_ContactRate,apc_rc_connected,rccalled,remindercallcontactrate,bucket1contacted,bucketcallcontacted,bucket1contactrate,Normalized_Greg,Normalized_denominator,Bucket1_Normalized
0,2024-04-22,2024-04-28,1925,3061,0.628879,3314,8464,0.391541,723,4343,0.166475,530,3472,0.15265


This SQL code seems to be calculating a normalization factor for a specific scenario related to loans and their bucket flow. Let's break down the logic step by step:

1. **Common Table Expressions (CTE): `dt`**
   - This CTE creates a temporary table (`dt`) with the start and end dates provided (`sd` and `ed`), used for filtering later.

2. **Common Table Expressions (CTE): `disb`**
   - This CTE retrieves distinct records from the `risk_credit_mis.loan_master_table` table, filtering for disbursement events that occurred on or before the provided end date (`WkendDate`).
   - The `rnk` column is calculated using the `ROW_NUMBER()` function partitioned by the last 10 digits of the mobile number.
   - The result is filtered to exclude records with a `loanPaidStatus` of 'Settled' or 'Completed'.

3. **Common Table Expressions (CTE): `cd`**
   - This CTE retrieves data from the `risk_credit_mis.customer_contact_details` table, including the customer ID and new mobile number, with row numbers assigned based on the change date.

4. **Common Table Expressions (CTE): `buck1`**
   - This CTE retrieves data related to loans in a specific DPD (Days Past Due) range from the `risk_credit_mis.loan_bucket_flow_report_core` table.
   - It joins with the previously defined `disb` CTE and includes additional information about the loan, customer, and mobile number.
   - The result is filtered to consider only the bucket date matching the start date (`WkstartDate`) and where the DPD is between 1 and 30.

5. **Common Table Expressions (CTE): `buck1end`**
   - This CTE refines the data from `buck1` by joining with the same bucket flow report table but considering the end date (`WkendDate`).
   - It includes information about the loan, customer, and mobile number.

6. **Main Query**
   - The main query calculates a normalization factor for a specific scenario related to loans and their bucket flow.
   - It counts the number of distinct loans where the DPD at the end date is less than or equal to the DPD at the start date (`Normalized_Greg`).
   - It also counts the total number of distinct loans at the start date (`Normalized_denominator`).
   - The final result includes the start and end dates, along with the calculated normalization factor (`Bucket1_Normalized`), which is the ratio of `Normalized_Greg` to `Normalized_denominator`.

Overall, this code is designed to analyze and report on the normalization factor for a specific scenario involving loans, their bucket flow, and the Days Past Due (DPD) values over a specified time period. The normalization factor provides insight into how many loans have maintained or improved their DPD status within the given time frame.

# Bucket 2 Contact Rate

In [10]:
sq = f"""WITH
  dt AS (
  SELECT
    DATE('{sd}') WkstartDate,
    DATE('{ed}') WkendDate),
disb as (select distinct disbursementDateTime, customerId, loanType, loanAccountNumber, SUBSTR(mobileNo, -10) as mobileno , loanPaidStatus
, row_number() over(partition by SUBSTR(mobileNo, -10) order by SUBSTR(mobileNo, -10))rnk
from `risk_credit_mis.loan_master_table`, dt 
where 
date_trunc(disbursementDateTime, day) <= dt.WkendDate
-- between dt.WkstartDate and dt.WkendDate 
and 
coalesce(loanPaidStatus, 'NA') not in ('Settled','Completed')
),
cd AS (
  SELECT
    customer_id,
    new_mobile_number,
    ROW_NUMBER() OVER(PARTITION BY customer_id ORDER BY change_date)rnk
  FROM
    `risk_credit_mis.customer_contact_details`
  WHERE
    ACTIVE = 'Y')
  ,
  buck2 as 
  (select distinct dt.WkstartDate, dt.WkendDate, lbfrc.loanAccountNumber, lbfrc.loanStatus, lbfrc.bucketDate, lbfrc.Max_current_DPD as DPD, lmt.customerId, right(cd.new_mobile_number, 10) newmobilenumber
  , row_number() over(partition by lbfrc.loanAccountNumber order by lbfrc.bucketDate desc) buck2rnk
  from `risk_credit_mis.loan_bucket_flow_report_core` lbfrc, dt 
  left join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = lbfrc.loanAccountNumber
  left join cd on cd.customer_id = lmt.customerId
  where date(bucketDate) between  date(dt.WkstartDate) and  date(dt.WkendDate)
  and coalesce(lbfrc.Max_current_DPD, 0) between 31 and 60 
  and date(lbfrc.firstDueDate) <= date(current_date())
  and lbfrc.loanStatus = 'In Arrears'
  and cd.rnk = 1
  )
    ,
buck2cr2 as 
(select distinct
 dt.WkstartDate,
  dt.WkendDate,
  g.Genesys_call_id,
  g.callDatetime,
  COALESCE(g.IS_APC, 0) APC,
  COALESCE(g.IS_RPC, 0) RPC,
  COALESCE(g.IS_PTP,0) PTP,
  RIGHT(g.mobileNumber, 10) mobileNumber,
  buck2.newmobilenumber mobilefrombuckettable,
  g.campaignName,
  g.agentGroup,
  g.agentFullName,
  g.employeeId,
  g.connected,
  g.notConnected,
  g.callResult, 
  buck2.loanAccountNumber,
  buck2.DPD,
  case when g.IS_APC = 1 then g.mobileNumber end apc_m,
  case when g.IS_RPC = 1 then g.mobileNumber end rpc_m,
  case when g.IS_PTP = 1 then mobileNumber end ptp_m,
  ROW_NUMBER() OVER(PARTITION BY g.Genesys_call_id ORDER BY g.callDatetime DESC, COALESCE(g.IS_APC, 0), COALESCE(g.IS_RPC, 0), COALESCE(g.IS_PTP, 0))rnk 
from  `risk_credit_mis.call_attempt_history_gensys` g,  dt
inner join buck2 on buck2.newmobilenumber = right(g.mobileNumber,10)
LEFT JOIN  `dap_ds_poweruser_playground.campaignmaster` cm ON  UPPER(cm.Campaignname) = UPPER(g.campaignName)
where cm.Reference = 'COLL' 
AND DATE(g.callDatetime) BETWEEN DATE(dt.WkstartDate) AND DATE(dt.WkendDate)
and buck2.buck2rnk = 1
)
,
buck2cr2base as 
(select buck2cr2.WkstartDate, buck2cr2.WkendDate, buck2cr2.Genesys_call_id, buck2cr2.callDatetime
, buck2cr2.mobileNumber
, buck2cr2.mobilefrombuckettable
, buck2cr2.campaignName
, buck2cr2.agentGroup
, buck2cr2.agentFullName
, buck2cr2.employeeId
, buck2cr2.connected
, buck2cr2.notConnected
, buck2cr2.callResult
, buck2cr2.loanAccountNumber
, buck2cr2.apc_m 
, buck2cr2.rpc_m
, buck2cr2.ptp_m 
, buck2cr2.DPD
, buck2cr2.rnk 
 from buck2cr2 where rnk = 1
)
Select * 
, (select count(distinct apc_m) from buck2cr2base) bucket2contacted,
(select count(distinct mobilefrombuckettable) Bucket2callcontactrate from buck2cr2base where mobilefrombuckettable is not null)bucket2callcontacted
from dt;
"""
dfbucket2contactrate = client.query(sq).to_dataframe()
dfbucket2contactrate['bucket2contactrate'] = dfbucket2contactrate['bucket2contacted']/dfbucket2contactrate['bucket2callcontacted']
dfbucket2contactrate

,WkstartDate,WkendDate,bucket2contacted,bucket2callcontacted,bucket2contactrate
0,2024-04-22,2024-04-28,177,2466,0.071776


In [11]:
df = pd.concat([df, dfbucket2contactrate[['bucket2contacted','bucket2callcontacted','bucket2contactrate']]], axis = 1)
df

,WkstartDate,WkendDate,apc_wc_connected,wccalled,WelcomeCall_ContactRate,apc_rc_connected,rccalled,remindercallcontactrate,bucket1contacted,bucketcallcontacted,bucket1contactrate,Normalized_Greg,Normalized_denominator,Bucket1_Normalized,bucket2contacted,bucket2callcontacted,bucket2contactrate
0,2024-04-22,2024-04-28,1925,3061,0.628879,3314,8464,0.391541,723,4343,0.166475,530,3472,0.15265,177,2466,0.071776


This SQL code appears to be querying data related to a specific type of call campaign, likely associated with loan collections or debt recovery, focusing on loans with Days Past Due (DPD) values between 31 and 60. Let's break down the logic step by step:

1. **Common Table Expressions (CTE): `dt`**
   - Similar to previous examples, this CTE creates a temporary table (`dt`) with the start and end dates provided (`sd` and `ed`), used for filtering later.

2. **Common Table Expressions (CTE): `disb`**
   - This CTE retrieves distinct records from the `risk_credit_mis.loan_master_table` table, filtering for disbursement events that occurred on or before the provided end date (`WkendDate`).
   - The `rnk` column is calculated using the `ROW_NUMBER()` function partitioned by the last 10 digits of the mobile number.
   - The result is filtered to exclude records with a `loanPaidStatus` of 'Settled' or 'Completed'.

3. **Common Table Expressions (CTE): `cd`**
   - This CTE retrieves data from the `risk_credit_mis.customer_contact_details` table, including the customer ID and new mobile number, with row numbers assigned based on the change date.

4. **Common Table Expressions (CTE): `buck2`**
   - This CTE retrieves data related to loans with DPD values between 31 and 60 from the `risk_credit_mis.loan_bucket_flow_report_core` table.
   - It joins with the previously defined `disb` CTE and includes additional information about the loan, customer, and mobile number.
   - The result is filtered to consider only the bucket date between the start and end dates, where the DPD is between 31 and 60.

5. **Common Table Expressions (CTE): `buck2cr2`**
   - This CTE retrieves call attempt data related to collections from the `risk_credit_mis.call_attempt_history_gensys` table.
   - It joins with the `buck2` CTE and includes details such as campaign information, call results, and row numbers based on the call datetime and flags for different call types (APC, RPC, PTP).
   - The result is filtered to include only records with `rnk = 1` and calls associated with the 'COLL' reference.

6. **Common Table Expressions (CTE): `buck2cr2base`**
   - This CTE is a copy of the previous `buck2cr2` CTE, retaining all columns.

7. **Main Query**
   - The main query selects various aggregated counts and values from the CTEs, providing metrics related to calls associated with loans in the specified DPD range.
   - The counts include the number of connected calls (`bucket2contacted`) and the total number of distinct mobile numbers contacted (`Bucket2callcontactrate`).
   - Finally, it calculates the contact rate as the ratio of connected calls to the total called numbers.

Overall, this code is designed to analyze and report metrics related to calls associated with loans in the specified DPD range (31 to 60), specifically focusing on connected calls and contact rates over a specified time period. The logic involves joining and filtering data from multiple tables to extract relevant information for analysis.

# Bucket 2 Normalization

In [12]:
sq = f"""WITH
  dt AS (
  SELECT
    DATE('{sd}') WkstartDate,
    DATE('{ed}') WkendDate),
disb as (select distinct disbursementDateTime, customerId, loanType, loanAccountNumber, SUBSTR(mobileNo, -10) as mobileno , loanPaidStatus
, row_number() over(partition by SUBSTR(mobileNo, -10) order by SUBSTR(mobileNo, -10))rnk
from `risk_credit_mis.loan_master_table`, dt 
where 
date_trunc(disbursementDateTime, day) <= dt.WkendDate
-- between dt.WkstartDate and dt.WkendDate 
and 
coalesce(loanPaidStatus, 'NA') not in ('Settled','Completed')
),
cd AS (
  SELECT
    customer_id,
    new_mobile_number,
    ROW_NUMBER() OVER(PARTITION BY customer_id ORDER BY change_date)rnk
  FROM
    `risk_credit_mis.customer_contact_details`
  WHERE
    ACTIVE = 'Y')
  ,
buck2 as 
(select distinct dt.WkstartDate, dt.WkendDate, lbfrc.bucketDate, lbfrc.loanAccountNumber, lbfrc.loanStatus, lbfrc.bucketDate, lbfrc.Max_current_DPD as DPD
  , lmt.customerId, right(cd.new_mobile_number, 10) newmobilenumber from `risk_credit_mis.loan_bucket_flow_report_core` lbfrc, dt 
  left join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = lbfrc.loanAccountNumber
  left join cd on cd.customer_id = lmt.customerId
  where date(bucketDate) =  date(dt.WkstartDate) and coalesce(lbfrc.Max_current_DPD, 0) between 31 and 60 ),
buck2end as
(select distinct dt.WkstartDate, dt.WkendDate, lbfrc.bucketDate, lbfrc.loanAccountNumber, lbfrc.loanStatus, lbfrc.bucketDate, lbfrc.Max_current_DPD as DPD
  , lmt.customerId, right(cd.new_mobile_number, 10) newmobilenumber 
from `risk_credit_mis.loan_bucket_flow_report_core` lbfrc, dt 
inner join buck2 on buck2.loanAccountNumber = lbfrc.loanAccountNumber
left join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = lbfrc.loanAccountNumber
left join cd on cd.customer_id = lmt.customerId
where date(lbfrc.bucketDate) =  date(dt.WkendDate))
select  dt.WkstartDate, dt.WkendDate
, count(distinct case when coalesce(buck2end.DPD, 0) <= coalesce(buck2.DPD, 0) then buck2.loanAccountNumber end) Normalized_buck2_Greg
, count(distinct buck2.loanAccountNumber) Normalized_buck2_denominator
from buck2, dt
left join buck2end on buck2end.loanAccountNumber = buck2.loanAccountNumber
group by 1, 2;
"""

dfbucket2normalization = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
dfbucket2normalization['bucket2Normalizationrate'] = dfbucket2normalization['Normalized_buck2_Greg']/dfbucket2normalization['Normalized_buck2_denominator']
dfbucket2normalization

Job ID b7f4d484-4ac9-48f8-8831-107ecc546063 successfully executed: |          |
Downloading: 100%|██████████|


,WkstartDate,WkendDate,Normalized_buck2_Greg,Normalized_buck2_denominator,bucket2Normalizationrate
0,2024-04-22,2024-04-28,101,2172,0.046501


In [13]:
dfbucket2normalization.columns

Index(['WkstartDate', 'WkendDate', 'Normalized_buck2_Greg',
       'Normalized_buck2_denominator', 'bucket2Normalizationrate'],
      dtype='object')

In [14]:
df = pd.concat([df,dfbucket2normalization[['Normalized_buck2_Greg',
       'Normalized_buck2_denominator', 'bucket2Normalizationrate']] ], axis = 1)
df

,WkstartDate,WkendDate,apc_wc_connected,wccalled,WelcomeCall_ContactRate,apc_rc_connected,rccalled,remindercallcontactrate,bucket1contacted,bucketcallcontacted,bucket1contactrate,Normalized_Greg,Normalized_denominator,Bucket1_Normalized,bucket2contacted,bucket2callcontacted,bucket2contactrate,Normalized_buck2_Greg,Normalized_buck2_denominator,bucket2Normalizationrate
0,2024-04-22,2024-04-28,1925,3061,0.628879,3314,8464,0.391541,723,4343,0.166475,530,3472,0.15265,177,2466,0.071776,101,2172,0.046501


This SQL code calculates a normalization rate for a specific scenario related to loans and their bucket flow, focusing on loans with Days Past Due (DPD) values between 31 and 60. Here's an explanation of the logic:

1. **Common Table Expressions (CTE): `dt`**
   - This CTE creates a temporary table (`dt`) with the start and end dates provided (`sd` and `ed`), which will be used for filtering later.

2. **Common Table Expressions (CTE): `disb`**
   - This CTE retrieves distinct records from the `risk_credit_mis.loan_master_table` table, filtering for disbursement events that occurred on or before the provided end date (`WkendDate`).
   - The `rnk` column is calculated using the `ROW_NUMBER()` function partitioned by the last 10 digits of the mobile number.
   - The result is filtered to exclude records with a `loanPaidStatus` of 'Settled' or 'Completed'.

3. **Common Table Expressions (CTE): `cd`**
   - This CTE retrieves data from the `risk_credit_mis.customer_contact_details` table, including the customer ID and new mobile number, with row numbers assigned based on the change date.

4. **Common Table Expressions (CTE): `buck2`**
   - This CTE retrieves data related to loans with DPD values between 31 and 60 from the `risk_credit_mis.loan_bucket_flow_report_core` table.
   - It joins with the previously defined `disb` CTE and includes additional information about the loan, customer, and mobile number.
   - The result is filtered to consider only the bucket date matching the start date (`WkstartDate`) and where the DPD is between 31 and 60.

5. **Common Table Expressions (CTE): `buck2end`**
   - This CTE refines the data from `buck2` by joining with the same bucket flow report table but considering the end date (`WkendDate`).
   - It includes information about the loan, customer, and mobile number.

6. **Main Query**
   - The main query calculates a normalization rate for loans with DPD values between 31 and 60.
   - It counts the number of distinct loans where the DPD at the end date is less than or equal to the DPD at the start date (`Normalized_buck2_Greg`).
   - It also counts the total number of distinct loans at the start date (`Normalized_buck2_denominator`).
   - The final result includes the start and end dates, along with the calculated normalization rate (`bucket2Normalizationrate`), which is the ratio of `Normalized_buck2_Greg` to `Normalized_buck2_denominator`.

Overall, this code is designed to analyze and report on the normalization rate for loans with DPD values between 31 and 60, providing insights into how many loans have maintained or improved their DPD status within the given time frame.

# Recovery Rate - Bucket1

In [15]:
sq = f"""
with Overdue as 
(select bucketDate
, DATE'{sd}' wkstartdate
,  DATE'{ed}' wkenddate
, loanStatus
, loanAccountNumber
,case  
  when coalesce(Max_current_DPD,0) <= 0 then 'B. Not Delinquent'
  when coalesce(Max_current_DPD,0) >= 1 and coalesce(Max_current_DPD, 0) <= 30 then 'C. 1-30DPD' 
  when coalesce(Max_current_DPD, 0) >=31 and coalesce(Max_current_DPD, 0) <= 60 then 'D. 31-60DPD'
  when coalesce(Max_current_DPD, 0) >=61 and coalesce(Max_current_DPD, 0) <= 90 then 'E. 61-90DPD'
  when coalesce(Max_current_DPD, 0) >=91 and coalesce(Max_current_DPD, 0) <= 120 then 'F. 91-120DPD'
  when coalesce(Max_current_DPD, 0) >=121 and coalesce(Max_current_DPD, 0) <= 150 then 'G. 121-150DPD'
  when coalesce(Max_current_DPD, 0) >=151 and coalesce(Max_current_DPD, 0) <= 180 then 'H. 151-180DPD'
  when coalesce(Max_current_DPD, 0) >=181 and coalesce(Max_current_DPD, 0) <= 360 then 'I. 181-360DPD'
  when coalesce(Max_current_DPD, 0) >=361 then 'J. >360DPD' end  bucketbin
,case
  when coalesce(Max_current_DPD,0) <= 0 then 0
  when coalesce(Max_current_DPD,0) >= 1 and coalesce(Max_current_DPD, 0) <= 30 then 1 
  when coalesce(Max_current_DPD, 0) >=31 and coalesce(Max_current_DPD, 0) <= 60 then 2
  when coalesce(Max_current_DPD, 0) >=61 and coalesce(Max_current_DPD, 0) <= 90 then 3
  when coalesce(Max_current_DPD, 0) >=91 and coalesce(Max_current_DPD, 0) <= 120 then 4
  when coalesce(Max_current_DPD, 0) >=121 and coalesce(Max_current_DPD, 0) <= 150 then 5
  when coalesce(Max_current_DPD, 0) >=151 and coalesce(Max_current_DPD, 0) <= 180 then 6
  when coalesce(Max_current_DPD, 0) >=181 and coalesce(Max_current_DPD, 0) <= 360 then 7
  when coalesce(Max_current_DPD, 0) >=361 then 8 end  bucket,
sum(Total_Outstanding_Amount) totaloutstandingamount,
sum(Total_Overdue_Amount) Total_Overdue_Amount
from `risk_credit_mis.loan_bucket_flow_report_core`
where bucketDate = DATE'{sd}'
and loanStatus in ('Normal', 'In Arrears')
and Total_Outstanding_Amount is not null
and Total_Outstanding_Amount > 0
group by 1, 2,3,4,5,6,7
)
,
overduebase as
(select bucketDate, wkstartdate,wkenddate, loanAccountNumber, bucket
, sum(totaloutstandingamount)totaloutstandingamount 
, sum(case when loanStatus = 'In Arrears' then totaloutstandingamount else 0 end) inArearstotaloutstandingamount
, sum(case when loanStatus = 'Normal' then totaloutstandingamount else 0 end) Normaltotaloutstandingamount
, sum(case when loanStatus = 'In Arrears' then Total_Overdue_Amount else 0 end) inArearstoTotal_Overdue_Amount
, sum(case when loanStatus = 'Normal' then Total_Overdue_Amount else 0 end) NormalTotal_Overdue_Amount
from Overdue
group by 1,2,3,4,5
)
,
payments as 
(select  DATE'{sd}' wkstartdate, DATE'{ed}' wkenddate,  loanAccountNumber, lastPaymentDate,isDelinquent, isCurrentDelinquent,
sum(installmentPaidAmount) payment
from `risk_credit_mis.loan_installments_table`
where coalesce(lastPaymentDate, date('3000-01-01')) between  DATE'{sd}' and DATE'{ed}'
and isDelinquent = 1
and coalesce(installmentPaidAmount, 0) > 0
group by 1,2,3,4,5,6
), 
paymentbase as  
(select wkstartdate, wkenddate, loanAccountNumber
, count(distinct lastPaymentDate) noofdayspaymentreceived 
, sum(case when isDelinquent = 1 and isCurrentDelinquent = 0 then 1 else 0 end) Normalized_cnt
, sum(case when isDelinquent = 1 and isCurrentDelinquent = 1 then 1 else 0 end) paymentreceivedbutstilldelinquent_cnt
, sum(case when isDelinquent = 1 and isCurrentDelinquent = 0 then payment else 0 end) Normalized
, sum(case when isDelinquent = 1 and isCurrentDelinquent = 1 then payment else 0 end) paymentreceivedbutstilldelinquent
, sum(payment) payment
from payments 
group by 1,2,3
)
select 
overduebase.bucketDate
, overduebase.wkstartdate
, overduebase.wkenddate
, bucket
, count(distinct overduebase.loanAccountNumber ) cntloanoverdue
, sum(overduebase.totaloutstandingamount) totaloutstandingamount
, sum(overduebase.inArearstotaloutstandingamount) inArearstotaloutstandingamount
, sum(overduebase.Normaltotaloutstandingamount) Normaltotaloutstandingamount
, sum(overduebase.inArearstoTotal_Overdue_Amount) inArearstoTotal_Overdue_Amount
, sum(overduebase.NormalTotal_Overdue_Amount)NormalTotal_Overdue_Amount
, count(distinct paymentbase.loanAccountNumber) cnt_loanpaymentsreceived
, max(noofdayspaymentreceived) noofdayspaymentreceived
, sum(Normalized_cnt) Normalized_cnt
, sum(Normalized)Normalized
, sum(paymentreceivedbutstilldelinquent_cnt)paymentreceivedbutstilldelinquent_cnt
, sum(paymentreceivedbutstilldelinquent) paymentreceivedbutstilldelinquent
, sum(payment)payment
from overduebase
left join paymentbase on overduebase.loanAccountNumber = paymentbase.loanAccountNumber 
group by 1,2,3,4

"""
dfrecovery = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfrecovery['paymentsfromoverdue'] = dfrecovery['Normalized'] + dfrecovery['paymentreceivedbutstilldelinquent']
dfrecovery['recoveryrate'] = (dfrecovery['Normalized'] + dfrecovery['paymentreceivedbutstilldelinquent'])/dfrecovery['inArearstoTotal_Overdue_Amount']
dfrecovery['paymentreceived'] = dfrecovery['Normalized'] + dfrecovery['paymentreceivedbutstilldelinquent']
dfrecovery

Job ID 5f319c89-3a70-4362-9566-0e5c2083322b successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,bucketDate,wkstartdate,wkenddate,bucket,cntloanoverdue,totaloutstandingamount,inArearstotaloutstandingamount,Normaltotaloutstandingamount,inArearstoTotal_Overdue_Amount,NormalTotal_Overdue_Amount,cnt_loanpaymentsreceived,noofdayspaymentreceived,Normalized_cnt,Normalized,paymentreceivedbutstilldelinquent_cnt,paymentreceivedbutstilldelinquent,payment,paymentsfromoverdue,recoveryrate,paymentreceived
0,2024-04-22,2024-04-22,2024-04-28,0,45540,6.002261e+08,0.000000e+00,6.002261e+08,0.000000e+00,307905.49,918,2,553,1181353.56,370,61547.32,1242900.88,1242900.88,inf,1242900.88
1,2024-04-22,2024-04-22,2024-04-28,2,2157,3.731925e+07,3.731925e+07,0.000000e+00,1.090436e+07,0.00,104,2,92,378862.84,58,59682.02,438544.86,438544.86,0.040217,438544.86
2,2024-04-22,2024-04-22,2024-04-28,7,7781,2.221316e+08,2.221316e+08,0.000000e+00,1.529429e+08,0.00,5,1,4,10159.17,4,7509.70,17668.87,17668.87,0.000116,17668.87
3,2024-04-22,2024-04-22,2024-04-28,5,1300,3.444752e+07,3.444752e+07,0.000000e+00,1.776342e+07,0.00,18,1,12,61567.42,15,27522.93,89090.35,89090.35,0.005015,89090.35
4,2024-04-22,2024-04-22,2024-04-28,8,7297,2.186286e+08,2.186286e+08,0.000000e+00,1.967054e+08,0.00,0,<NA>,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,NaN
5,2024-04-22,2024-04-22,2024-04-28,1,3424,5.418962e+07,5.418962e+07,0.000000e+00,8.536883e+06,0.00,563,2,500,1134073.77,103,63467.18,1197540.95,1197540.95,0.140278,1197540.95
6,2024-04-22,2024-04-22,2024-04-28,3,1782,3.450635e+07,3.450635e+07,0.000000e+00,1.364885e+07,0.00,30,2,23,116486.53,22,43472.53,159959.06,159959.06,0.011720,159959.06
7,2024-04-22,2024-04-22,2024-04-28,6,1530,4.113479e+07,4.113479e+07,0.000000e+00,2.504882e+07,0.00,15,1,12,54673.79,14,21462.34,76136.13,76136.13,0.003040,76136.13
8,2024-04-22,2024-04-22,2024-04-28,4,1372,3.027035e+07,3.027035e+07,0.000000e+00,1.442353e+07,0.00,23,1,17,109722.60,17,22087.31,131809.91,131809.91,0.009139,131809.91


In [16]:
dfr = dfrecovery[dfrecovery['bucket'] == 1].copy()
dfr= dfr[['wkstartdate','wkenddate','inArearstoTotal_Overdue_Amount','paymentsfromoverdue','recoveryrate']]
dfr.rename(columns ={'wkstartdate':'WkstartDate','wkenddate': 'WkendDate', 'inArearstoTotal_Overdue_Amount':'Bucket1_inArearstoTotal_Overdue_Amount', 'paymentsfromoverdue':'Bucket1_paymentsfromoverdue', 'recoveryrate':'Bucket1_recoveryrate'}, inplace = True)
dfr.columns

Index(['WkstartDate', 'WkendDate', 'Bucket1_inArearstoTotal_Overdue_Amount',
       'Bucket1_paymentsfromoverdue', 'Bucket1_recoveryrate'],
      dtype='object')

In [17]:
dfr

,WkstartDate,WkendDate,Bucket1_inArearstoTotal_Overdue_Amount,Bucket1_paymentsfromoverdue,Bucket1_recoveryrate
5,2024-04-22,2024-04-28,8536882.89,1197540.95,0.140278


In [18]:
df

,WkstartDate,WkendDate,apc_wc_connected,wccalled,WelcomeCall_ContactRate,apc_rc_connected,rccalled,remindercallcontactrate,bucket1contacted,bucketcallcontacted,bucket1contactrate,Normalized_Greg,Normalized_denominator,Bucket1_Normalized,bucket2contacted,bucket2callcontacted,bucket2contactrate,Normalized_buck2_Greg,Normalized_buck2_denominator,bucket2Normalizationrate
0,2024-04-22,2024-04-28,1925,3061,0.628879,3314,8464,0.391541,723,4343,0.166475,530,3472,0.15265,177,2466,0.071776,101,2172,0.046501


In [19]:
dfr.reset_index(drop=True, inplace=True)
df = pd.concat([df, dfr[['Bucket1_inArearstoTotal_Overdue_Amount', 'Bucket1_paymentsfromoverdue','Bucket1_recoveryrate']]], axis = 1)
df

,WkstartDate,WkendDate,apc_wc_connected,wccalled,WelcomeCall_ContactRate,apc_rc_connected,rccalled,remindercallcontactrate,bucket1contacted,bucketcallcontacted,...,Bucket1_Normalized,bucket2contacted,bucket2callcontacted,bucket2contactrate,Normalized_buck2_Greg,Normalized_buck2_denominator,bucket2Normalizationrate,Bucket1_inArearstoTotal_Overdue_Amount,Bucket1_paymentsfromoverdue,Bucket1_recoveryrate
0,2024-04-22,2024-04-28,1925,3061,0.628879,3314,8464,0.391541,723,4343,...,0.15265,177,2466,0.071776,101,2172,0.046501,8536882.89,1197540.95,0.140278


# Recovery Rate - Bucket2

In [20]:
dfr = dfrecovery[dfrecovery['bucket'] == 2].copy()
dfr= dfr[['wkstartdate','wkenddate','inArearstoTotal_Overdue_Amount','paymentsfromoverdue','recoveryrate']]
dfr.rename(columns ={'wkstartdate':'WkstartDate','wkenddate': 'WkendDate', 'inArearstoTotal_Overdue_Amount':'Bucket2_inArearstoTotal_Overdue_Amount', 'paymentsfromoverdue':'Bucket2_paymentsfromoverdue', 'recoveryrate':'Bucket2_recoveryrate'}, inplace = True)
dfr.columns

Index(['WkstartDate', 'WkendDate', 'Bucket2_inArearstoTotal_Overdue_Amount',
       'Bucket2_paymentsfromoverdue', 'Bucket2_recoveryrate'],
      dtype='object')

In [21]:
dfr.reset_index(drop=True, inplace=True)
df = pd.concat([df, dfr[['Bucket2_inArearstoTotal_Overdue_Amount', 'Bucket2_paymentsfromoverdue','Bucket2_recoveryrate']]], axis = 1)
df

,WkstartDate,WkendDate,apc_wc_connected,wccalled,WelcomeCall_ContactRate,apc_rc_connected,rccalled,remindercallcontactrate,bucket1contacted,bucketcallcontacted,...,bucket2contactrate,Normalized_buck2_Greg,Normalized_buck2_denominator,bucket2Normalizationrate,Bucket1_inArearstoTotal_Overdue_Amount,Bucket1_paymentsfromoverdue,Bucket1_recoveryrate,Bucket2_inArearstoTotal_Overdue_Amount,Bucket2_paymentsfromoverdue,Bucket2_recoveryrate
0,2024-04-22,2024-04-28,1925,3061,0.628879,3314,8464,0.391541,723,4343,...,0.071776,101,2172,0.046501,8536882.89,1197540.95,0.140278,10904360.07,438544.86,0.040217


# Recovery Rate - Bucket3

In [22]:
dfr = dfrecovery[dfrecovery['bucket'] == 3].copy()
dfr= dfr[['wkstartdate','wkenddate','inArearstoTotal_Overdue_Amount','paymentsfromoverdue','recoveryrate']]
dfr.rename(columns ={'wkstartdate':'WkstartDate','wkenddate': 'WkendDate',
                     'inArearstoTotal_Overdue_Amount':'Bucket3_inArearstoTotal_Overdue_Amount',
                     'paymentsfromoverdue':'Bucket3_paymentsfromoverdue',
                     'recoveryrate':'Bucket3_recoveryrate'}, inplace = True)
dfr.columns

Index(['WkstartDate', 'WkendDate', 'Bucket3_inArearstoTotal_Overdue_Amount',
       'Bucket3_paymentsfromoverdue', 'Bucket3_recoveryrate'],
      dtype='object')

In [23]:
dfr.reset_index(drop=True, inplace=True)
df = pd.concat([df, dfr[['Bucket3_inArearstoTotal_Overdue_Amount',
       'Bucket3_paymentsfromoverdue', 'Bucket3_recoveryrate']]], axis = 1)
df

,WkstartDate,WkendDate,apc_wc_connected,wccalled,WelcomeCall_ContactRate,apc_rc_connected,rccalled,remindercallcontactrate,bucket1contacted,bucketcallcontacted,...,bucket2Normalizationrate,Bucket1_inArearstoTotal_Overdue_Amount,Bucket1_paymentsfromoverdue,Bucket1_recoveryrate,Bucket2_inArearstoTotal_Overdue_Amount,Bucket2_paymentsfromoverdue,Bucket2_recoveryrate,Bucket3_inArearstoTotal_Overdue_Amount,Bucket3_paymentsfromoverdue,Bucket3_recoveryrate
0,2024-04-22,2024-04-28,1925,3061,0.628879,3314,8464,0.391541,723,4343,...,0.046501,8536882.89,1197540.95,0.140278,10904360.07,438544.86,0.040217,13648845.42,159959.06,0.01172


# Recovery Rate - Bucket4

In [24]:
dfr = dfrecovery[dfrecovery['bucket'] == 4].copy()
dfr= dfr[['wkstartdate','wkenddate','inArearstoTotal_Overdue_Amount','paymentsfromoverdue','recoveryrate']]
dfr.rename(columns ={'wkstartdate':'WkstartDate','wkenddate': 'WkendDate',
                     'inArearstoTotal_Overdue_Amount':'Bucket4_inArearstoTotal_Overdue_Amount',
                     'paymentsfromoverdue':'Bucket4_paymentsfromoverdue',
                     'recoveryrate':'Bucket4_recoveryrate'}, inplace = True)
dfr.columns

Index(['WkstartDate', 'WkendDate', 'Bucket4_inArearstoTotal_Overdue_Amount',
       'Bucket4_paymentsfromoverdue', 'Bucket4_recoveryrate'],
      dtype='object')

In [25]:
dfr.reset_index(drop=True, inplace=True)
df = pd.concat([df, dfr[['Bucket4_inArearstoTotal_Overdue_Amount',
       'Bucket4_paymentsfromoverdue', 'Bucket4_recoveryrate']]], axis = 1)
df

,WkstartDate,WkendDate,apc_wc_connected,wccalled,WelcomeCall_ContactRate,apc_rc_connected,rccalled,remindercallcontactrate,bucket1contacted,bucketcallcontacted,...,Bucket1_recoveryrate,Bucket2_inArearstoTotal_Overdue_Amount,Bucket2_paymentsfromoverdue,Bucket2_recoveryrate,Bucket3_inArearstoTotal_Overdue_Amount,Bucket3_paymentsfromoverdue,Bucket3_recoveryrate,Bucket4_inArearstoTotal_Overdue_Amount,Bucket4_paymentsfromoverdue,Bucket4_recoveryrate
0,2024-04-22,2024-04-28,1925,3061,0.628879,3314,8464,0.391541,723,4343,...,0.140278,10904360.07,438544.86,0.040217,13648845.42,159959.06,0.01172,14423530.02,131809.91,0.009139


# Recovery Rate - Bucket5

In [26]:
dfr = dfrecovery[dfrecovery['bucket'] == 5].copy()
dfr= dfr[['wkstartdate','wkenddate','inArearstoTotal_Overdue_Amount','paymentsfromoverdue','recoveryrate']]
dfr.rename(columns ={'wkstartdate':'WkstartDate','wkenddate': 'WkendDate',
                     'inArearstoTotal_Overdue_Amount':'Bucket5_inArearstoTotal_Overdue_Amount',
                     'paymentsfromoverdue':'Bucket5_paymentsfromoverdue',
                     'recoveryrate':'Bucket5_recoveryrate'}, inplace = True)
dfr.columns

Index(['WkstartDate', 'WkendDate', 'Bucket5_inArearstoTotal_Overdue_Amount',
       'Bucket5_paymentsfromoverdue', 'Bucket5_recoveryrate'],
      dtype='object')

In [27]:
dfr.reset_index(drop=True, inplace=True)
df = pd.concat([df, dfr[['Bucket5_inArearstoTotal_Overdue_Amount',
       'Bucket5_paymentsfromoverdue', 'Bucket5_recoveryrate']]], axis = 1)
df

,WkstartDate,WkendDate,apc_wc_connected,wccalled,WelcomeCall_ContactRate,apc_rc_connected,rccalled,remindercallcontactrate,bucket1contacted,bucketcallcontacted,...,Bucket2_recoveryrate,Bucket3_inArearstoTotal_Overdue_Amount,Bucket3_paymentsfromoverdue,Bucket3_recoveryrate,Bucket4_inArearstoTotal_Overdue_Amount,Bucket4_paymentsfromoverdue,Bucket4_recoveryrate,Bucket5_inArearstoTotal_Overdue_Amount,Bucket5_paymentsfromoverdue,Bucket5_recoveryrate
0,2024-04-22,2024-04-28,1925,3061,0.628879,3314,8464,0.391541,723,4343,...,0.040217,13648845.42,159959.06,0.01172,14423530.02,131809.91,0.009139,17763422.98,89090.35,0.005015


# Recovery Rate - Bucket6

In [28]:
dfr = dfrecovery[dfrecovery['bucket'] == 6].copy()
dfr= dfr[['wkstartdate','wkenddate','inArearstoTotal_Overdue_Amount','paymentsfromoverdue','recoveryrate']]
dfr.rename(columns ={'wkstartdate':'WkstartDate','wkenddate': 'WkendDate',
                     'inArearstoTotal_Overdue_Amount':'Bucket6_inArearstoTotal_Overdue_Amount',
                     'paymentsfromoverdue':'Bucket6_paymentsfromoverdue',
                     'recoveryrate':'Bucket6_recoveryrate'}, inplace = True)
dfr.columns

Index(['WkstartDate', 'WkendDate', 'Bucket6_inArearstoTotal_Overdue_Amount',
       'Bucket6_paymentsfromoverdue', 'Bucket6_recoveryrate'],
      dtype='object')

In [29]:
dfr.reset_index(drop=True, inplace=True)
df = pd.concat([df, dfr[['Bucket6_inArearstoTotal_Overdue_Amount',
       'Bucket6_paymentsfromoverdue', 'Bucket6_recoveryrate']]], axis = 1)
df

,WkstartDate,WkendDate,apc_wc_connected,wccalled,WelcomeCall_ContactRate,apc_rc_connected,rccalled,remindercallcontactrate,bucket1contacted,bucketcallcontacted,...,Bucket3_recoveryrate,Bucket4_inArearstoTotal_Overdue_Amount,Bucket4_paymentsfromoverdue,Bucket4_recoveryrate,Bucket5_inArearstoTotal_Overdue_Amount,Bucket5_paymentsfromoverdue,Bucket5_recoveryrate,Bucket6_inArearstoTotal_Overdue_Amount,Bucket6_paymentsfromoverdue,Bucket6_recoveryrate
0,2024-04-22,2024-04-28,1925,3061,0.628879,3314,8464,0.391541,723,4343,...,0.01172,14423530.02,131809.91,0.009139,17763422.98,89090.35,0.005015,25048815.96,76136.13,0.00304


# Recovery Rate - Bucket7

In [30]:
dfr = dfrecovery[dfrecovery['bucket'] == 7].copy()
dfr= dfr[['wkstartdate','wkenddate','inArearstoTotal_Overdue_Amount','paymentsfromoverdue','recoveryrate']]
dfr.rename(columns ={'wkstartdate':'WkstartDate','wkenddate': 'WkendDate',
                     'inArearstoTotal_Overdue_Amount':'Bucket7_inArearstoTotal_Overdue_Amount',
                     'paymentsfromoverdue':'Bucket7_paymentsfromoverdue',
                     'recoveryrate':'Bucket7_recoveryrate'}, inplace = True)
dfr.columns

Index(['WkstartDate', 'WkendDate', 'Bucket7_inArearstoTotal_Overdue_Amount',
       'Bucket7_paymentsfromoverdue', 'Bucket7_recoveryrate'],
      dtype='object')

In [31]:
dfr.reset_index(drop=True, inplace=True)
df = pd.concat([df, dfr[['Bucket7_inArearstoTotal_Overdue_Amount',
       'Bucket7_paymentsfromoverdue', 'Bucket7_recoveryrate']]], axis = 1)
df

,WkstartDate,WkendDate,apc_wc_connected,wccalled,WelcomeCall_ContactRate,apc_rc_connected,rccalled,remindercallcontactrate,bucket1contacted,bucketcallcontacted,...,Bucket4_recoveryrate,Bucket5_inArearstoTotal_Overdue_Amount,Bucket5_paymentsfromoverdue,Bucket5_recoveryrate,Bucket6_inArearstoTotal_Overdue_Amount,Bucket6_paymentsfromoverdue,Bucket6_recoveryrate,Bucket7_inArearstoTotal_Overdue_Amount,Bucket7_paymentsfromoverdue,Bucket7_recoveryrate
0,2024-04-22,2024-04-28,1925,3061,0.628879,3314,8464,0.391541,723,4343,...,0.009139,17763422.98,89090.35,0.005015,25048815.96,76136.13,0.00304,1.529429e+08,17668.87,0.000116


# Recovery Rate - Bucket8

In [32]:
dfr = dfrecovery[dfrecovery['bucket'] == 8].copy()
dfr= dfr[['wkstartdate','wkenddate','inArearstoTotal_Overdue_Amount','paymentsfromoverdue','recoveryrate']]
dfr.rename(columns ={'wkstartdate':'WkstartDate','wkenddate': 'WkendDate',
                     'inArearstoTotal_Overdue_Amount':'Bucket8_inArearstoTotal_Overdue_Amount',
                     'paymentsfromoverdue':'Bucket8_paymentsfromoverdue',
                     'recoveryrate':'Bucket8_recoveryrate'}, inplace = True)
dfr.columns

Index(['WkstartDate', 'WkendDate', 'Bucket8_inArearstoTotal_Overdue_Amount',
       'Bucket8_paymentsfromoverdue', 'Bucket8_recoveryrate'],
      dtype='object')

In [33]:
dfr.reset_index(drop=True, inplace=True)
df = pd.concat([df, dfr[['Bucket8_inArearstoTotal_Overdue_Amount',
       'Bucket8_paymentsfromoverdue', 'Bucket8_recoveryrate']]], axis = 1)
df

,WkstartDate,WkendDate,apc_wc_connected,wccalled,WelcomeCall_ContactRate,apc_rc_connected,rccalled,remindercallcontactrate,bucket1contacted,bucketcallcontacted,...,Bucket5_recoveryrate,Bucket6_inArearstoTotal_Overdue_Amount,Bucket6_paymentsfromoverdue,Bucket6_recoveryrate,Bucket7_inArearstoTotal_Overdue_Amount,Bucket7_paymentsfromoverdue,Bucket7_recoveryrate,Bucket8_inArearstoTotal_Overdue_Amount,Bucket8_paymentsfromoverdue,Bucket8_recoveryrate
0,2024-04-22,2024-04-28,1925,3061,0.628879,3314,8464,0.391541,723,4343,...,0.005015,25048815.96,76136.13,0.00304,1.529429e+08,17668.87,0.000116,196705449.0,NaN,NaN


# Recovery Rate

In [34]:
sq = f"""with Overdue as 
(select bucketDate
, DATE('{sd}') wkstartdate
,  DATE('{ed}') wkenddate
, loanStatus, loanAccountNumber,
sum(Total_Outstanding_Amount) totaloutstandingamount,
sum(Total_Overdue_Amount) Total_Overdue_Amount
from `risk_credit_mis.loan_bucket_flow_report_core`
where bucketDate = DATE('{sd}')
and loanStatus in ('Normal', 'In Arrears')
and Total_Outstanding_Amount is not null
and Total_Outstanding_Amount > 0
group by 1, 2,3,4,5
),
overduebase as
(select bucketDate, wkstartdate,wkenddate, loanAccountNumber, sum(totaloutstandingamount)totaloutstandingamount 
, sum(case when loanStatus = 'In Arrears' then totaloutstandingamount else 0 end) inArearstotaloutstandingamount
, sum(case when loanStatus = 'Normal' then totaloutstandingamount else 0 end) Normaltotaloutstandingamount
, sum(case when loanStatus = 'In Arrears' then Total_Overdue_Amount else 0 end) inArearstoTotal_Overdue_Amount
, sum(case when loanStatus = 'Normal' then Total_Overdue_Amount else 0 end) NormalTotal_Overdue_Amount
from Overdue
group by 1,2,3,4
),
payments as 
(select  DATE('{sd}') wkstartdate, DATE('{ed}') wkenddate,  loanAccountNumber, lastPaymentDate,isDelinquent, isCurrentDelinquent,
sum(installmentPaidAmount) payment
from `risk_credit_mis.loan_installments_table`
where coalesce(lastPaymentDate, date('3000-01-01')) between  DATE('{sd}') and DATE('{ed}')
and isDelinquent = 1
and coalesce(installmentPaidAmount, 0) > 0
group by 1,2,3,4,5,6
), 
paymentbase as  
(select wkstartdate, wkenddate, loanAccountNumber
, count(distinct lastPaymentDate) noofdayspaymentreceived 
, sum(case when isDelinquent = 1 and isCurrentDelinquent = 0 then 1 else 0 end) Normalized_cnt
, sum(case when isDelinquent = 1 and isCurrentDelinquent = 1 then 1 else 0 end) paymentreceivedbutstilldelinquent_cnt
, sum(case when isDelinquent = 1 and isCurrentDelinquent = 0 then payment else 0 end) Normalized
, sum(case when isDelinquent = 1 and isCurrentDelinquent = 1 then payment else 0 end) paymentreceivedbutstilldelinquent
, sum(payment) payment
from payments 
group by 1,2,3
)
select 
overduebase.bucketDate
, overduebase.wkstartdate
, overduebase.wkenddate
, count(distinct overduebase.loanAccountNumber ) cntloanoverdue
, sum(overduebase.totaloutstandingamount) totaloutstandingamount
, sum(overduebase.inArearstotaloutstandingamount) inArearstotaloutstandingamount
, sum(overduebase.Normaltotaloutstandingamount) Normaltotaloutstandingamount
, sum(overduebase.inArearstoTotal_Overdue_Amount) inArearstoTotal_Overdue_Amount
, sum(overduebase.NormalTotal_Overdue_Amount)NormalTotal_Overdue_Amount
, count(distinct paymentbase.loanAccountNumber) cnt_loanpaymentsreceived
, max(noofdayspaymentreceived) noofdayspaymentreceived
, sum(Normalized_cnt) Normalized_cnt
, sum(Normalized)Normalized
, sum(paymentreceivedbutstilldelinquent_cnt)paymentreceivedbutstilldelinquent_cnt
, sum(paymentreceivedbutstilldelinquent) paymentreceivedbutstilldelinquent
, sum(payment)payment
from overduebase
left join paymentbase on overduebase.loanAccountNumber = paymentbase.loanAccountNumber 
group by 1,2,3
"""
dfrecovery = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfrecovery['paymentsfromoverdue'] = dfrecovery['Normalized'] + dfrecovery['paymentreceivedbutstilldelinquent']
dfrecovery['recoveryrate'] = (dfrecovery['Normalized'] + dfrecovery['paymentreceivedbutstilldelinquent'])/dfrecovery['inArearstoTotal_Overdue_Amount']
dfrecovery

Job ID 96fcdb39-855d-4192-a145-9fc3bfc87984 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,bucketDate,wkstartdate,wkenddate,cntloanoverdue,totaloutstandingamount,inArearstotaloutstandingamount,Normaltotaloutstandingamount,inArearstoTotal_Overdue_Amount,NormalTotal_Overdue_Amount,cnt_loanpaymentsreceived,noofdayspaymentreceived,Normalized_cnt,Normalized,paymentreceivedbutstilldelinquent_cnt,paymentreceivedbutstilldelinquent,payment,paymentsfromoverdue,recoveryrate
0,2024-04-22,2024-04-22,2024-04-28,72183,1.272854e+09,6.726281e+08,6.002261e+08,4.399742e+08,307905.49,1676,2,1213,3046899.68,603,306751.33,3353651.01,3353651.01,0.007622


In [35]:
dfrecovery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 18 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   bucketDate                             1 non-null      dbdate 
 1   wkstartdate                            1 non-null      dbdate 
 2   wkenddate                              1 non-null      dbdate 
 3   cntloanoverdue                         1 non-null      Int64  
 4   totaloutstandingamount                 1 non-null      float64
 5   inArearstotaloutstandingamount         1 non-null      float64
 6   Normaltotaloutstandingamount           1 non-null      float64
 7   inArearstoTotal_Overdue_Amount         1 non-null      float64
 8   NormalTotal_Overdue_Amount             1 non-null      float64
 9   cnt_loanpaymentsreceived               1 non-null      Int64  
 10  noofdayspaymentreceived                1 non-null      Int64  
 11  Normalized

In [36]:
df = pd.concat([df, dfrecovery[['inArearstoTotal_Overdue_Amount','paymentsfromoverdue','recoveryrate']]], axis = 1)
df

,WkstartDate,WkendDate,apc_wc_connected,wccalled,WelcomeCall_ContactRate,apc_rc_connected,rccalled,remindercallcontactrate,bucket1contacted,bucketcallcontacted,...,Bucket6_recoveryrate,Bucket7_inArearstoTotal_Overdue_Amount,Bucket7_paymentsfromoverdue,Bucket7_recoveryrate,Bucket8_inArearstoTotal_Overdue_Amount,Bucket8_paymentsfromoverdue,Bucket8_recoveryrate,inArearstoTotal_Overdue_Amount,paymentsfromoverdue,recoveryrate
0,2024-04-22,2024-04-28,1925,3061,0.628879,3314,8464,0.391541,723,4343,...,0.00304,1.529429e+08,17668.87,0.000116,196705449.0,NaN,NaN,4.399742e+08,3353651.01,0.007622


This SQL code calculates recovery-related metrics for overdue loans. Here's an explanation of the logic:

1. **Common Table Expressions (CTE): `Overdue`**
   - This CTE selects relevant information from the `risk_credit_mis.loan_bucket_flow_report_core` table for loans that are either in a 'Normal' or 'In Arrears' status on the provided start date (`wkstartdate`).
   - The data is filtered to include only loans with positive and non-null outstanding amounts.
   - The result is grouped by bucket date, start date, end date (`wkenddate`), loan status, and loan account number, and aggregated sums are calculated for total outstanding amounts and total overdue amounts.

2. **Common Table Expressions (CTE): `overduebase`**
   - This CTE refines the data from the `Overdue` CTE.
   - It aggregates the data further, grouping by bucket date, start date, end date, and loan account number.
   - The result includes aggregated sums for total outstanding amounts, total outstanding amounts for loans in arrears, total outstanding amounts for normal loans, total overdue amounts for loans in arrears, and total overdue amounts for normal loans.

3. **Common Table Expressions (CTE): `payments`**
   - This CTE selects information from the `risk_credit_mis.loan_installments_table` related to loan payments.
   - It filters for delinquent loans with positive installment paid amounts between the provided start and end dates.
   - The result is grouped by start date, end date, loan account number, last payment date, isDelinquent, and isCurrentDelinquent, and aggregated sums are calculated for the payment amount.

4. **Common Table Expressions (CTE): `paymentbase`**
   - This CTE refines the data from the `payments` CTE.
   - It aggregates the data further, grouping by start date, end date, and loan account number.
   - The result includes counts and sums related to the number of days payments were received, the number of normalized payments, the number of payments received but still delinquent, and the total payment amount.

5. **Main Query**
   - The main query brings together the data from `overduebase` and `paymentbase`, joining them on the loan account number.
   - The final result includes aggregated metrics such as the count of overdue loans, total outstanding amounts, total overdue amounts, the count of loans with payments received, the maximum number of days payments were received, counts and sums related to normalized payments and payments received but still delinquent, and the total payment amount.
   - A new column, `paymentsfromoverdue`, is calculated as the sum of normalized payments and payments received but still delinquent.
   - Another new column, `recoveryrate`, is calculated as the ratio of `paymentsfromoverdue` to `inArearstoTotal_Overdue_Amount`.

In summary, the code provides insights into the recovery status of overdue loans, considering outstanding amounts, overdue amounts, and payments received within a specified time frame. The recovery rate is also calculated as an important metric for assessing the effectiveness of recovery efforts.

# Roll-rate Normal into Bucket 1

In [37]:
sq = f"""
with base as 
(select 
DATE('{sd}') wkstartdate
, DATE('{ed}') wkenddate
, loanAccountNumber
, installmentDueDate 
, DPDwoToleranceCustom dpd
, isDelinquent
, isCurrentDelinquent
, installmentAmount
, installmentPaidAmount
from `risk_credit_mis.loan_installments_table`
where date(installmentDueDate) between DATE('{sd}') and DATE('{ed}')
)
select wkstartdate, wkenddate 
, count(distinct loanAccountNumber) cntloans
, min(dpd) mindpd
, max(dpd) maxdpd
, sum(case when coalesce(dpd, 0) > 0 then isDelinquent else 0 end) wentdelinquent
, sum(case when coalesce(dpd, 0) > 0 and isDelinquent = 1 then isCurrentDelinquent else 0 end) stilldelinquent
, sum(case when coalesce(dpd, 0) between 1 and 30 then isDelinquent else 0 end) - sum(case when coalesce(dpd, 0) between 1 and 30 and isDelinquent = 1 then isCurrentDelinquent else 0 end) rollbacked
, sum(case when coalesce(dpd, 0) > 0 and isDelinquent = 1 then installmentAmount else 0 end) defaulteddelinquentamount
, sum(case when coalesce(dpd, 0) > 0 and isDelinquent = 1 and isCurrentDelinquent = 1 then installmentAmount else 0 end) stilldelinquentamount
, sum(case when coalesce(dpd, 0) between 1 and 30 and isDelinquent = 1 and isCurrentDelinquent = 0 then installmentPaidAmount else 0 end) rollbacked_amount
from base 
group by 1,2
;"""

dfrollrate = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfrollrate['rollback_rate'] =  dfrollrate['rollbacked']/dfrollrate['wentdelinquent']
dfrollrate['rollback_rate_v'] = dfrollrate['rollbacked_amount']/dfrollrate['defaulteddelinquentamount']
dfrollrate

Job ID f70bfb5b-2d56-47c1-a774-ce33591a07e1 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,wkstartdate,wkenddate,cntloans,mindpd,maxdpd,wentdelinquent,stilldelinquent,rollbacked,defaulteddelinquentamount,stilldelinquentamount,rollbacked_amount,rollback_rate,rollback_rate_v
0,2024-04-22,2024-04-28,8868,0,6,2884,2454,430,6278667.8,5405507.84,892494.13,0.149098,0.142147


In [38]:
dfrollrate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   wkstartdate                1 non-null      dbdate 
 1   wkenddate                  1 non-null      dbdate 
 2   cntloans                   1 non-null      Int64  
 3   mindpd                     1 non-null      Int64  
 4   maxdpd                     1 non-null      Int64  
 5   wentdelinquent             1 non-null      Int64  
 6   stilldelinquent            1 non-null      Int64  
 7   rollbacked                 1 non-null      Int64  
 8   defaulteddelinquentamount  1 non-null      float64
 9   stilldelinquentamount      1 non-null      float64
 10  rollbacked_amount          1 non-null      float64
 11  rollback_rate              1 non-null      Float64
 12  rollback_rate_v            1 non-null      float64
dtypes: Float64(1), Int64(6), dbdate(2), float64(4)
memory 

In [39]:
df = pd.concat([df, dfrollrate[['wentdelinquent','rollbacked', 'defaulteddelinquentamount', 'rollbacked_amount', 'rollback_rate', 'rollback_rate_v']]], axis = 1)

This SQL code calculates delinquency-related metrics for loans within a specified date range. Here's an explanation of the logic:

1. **Common Table Expressions (CTE): `base`**
   - This CTE selects relevant information from the `risk_credit_mis.loan_installments_table` table, including loan account number, installment due date, Days Past Due (DPD) without tolerance, delinquency indicators (`isDelinquent` and `isCurrentDelinquent`), installment amount, and installment paid amount.
   - The data is filtered to include only installment due dates falling within the provided start and end dates.

2. **Main Query**
   - The main query aggregates the data from the `base` CTE.
   - It calculates various metrics related to delinquency, including the count of loans (`cntloans`), the minimum and maximum DPD values (`mindpd` and `maxdpd`), the count of loans that went delinquent (`wentdelinquent`), the count of loans still delinquent (`stilldelinquent`), the count of loans rolled back from delinquency (`rollbacked`), and the sum of defaulted delinquent amounts (`defaulteddelinquentamount`).
   - It also calculates the sum of installment amounts for loans still delinquent (`stilldelinquentamount`) and the sum of installment amounts for loans rolled back from delinquency (`rollbacked_amount`).
   - Two new columns are created: `rollback_rate` (the ratio of loans rolled back to loans that went delinquent) and `rollback_rate_v` (the ratio of the sum of installment amounts for loans rolled back to the sum of defaulted delinquent amounts).

In summary, the code provides insights into the delinquency status of loans, including the count of delinquent loans, the DPD distribution, and metrics related to loans that went delinquent, loans still delinquent, and loans rolled back from delinquency. The rollback rates are calculated to assess the effectiveness of rollback efforts.m

# Other details

In [40]:
sq = f"""With
  dt AS (
  SELECT
    DATE('{sd}') WkstartDate,
    DATE('{ed}') WkendDate),
disb as (select distinct disbursementDateTime, customerId, loanType, loanAccountNumber, SUBSTR(mobileNo, -10) as mobileno , loanPaidStatus
, row_number() over(partition by SUBSTR(mobileNo, -10) order by SUBSTR(mobileNo, -10))rnk
from `risk_credit_mis.loan_master_table`, dt 
where 
date_trunc(disbursementDateTime, day) <= dt.WkendDate
-- between dt.WkstartDate and dt.WkendDate 
and 
coalesce(loanPaidStatus, 'NA') not in ('Settled','Completed')
)
,
wc1 as 
(
SELECT
  dt.WkstartDate,
  dt.WkendDate,
  Genesys_call_id,
  callDatetime,
  COALESCE(IS_APC, 0) APC,
  COALESCE(IS_RPC, 0) RPC,
  COALESCE(IS_PTP,0) PTP,
  RIGHT(mobileNumber, 10) mobileNumber,
  g.campaignName,
  g.agentGroup,
  g.agentFullName,
  g.employeeId,
  g.connected,
  g.notConnected,
  g.callResult,
  ROW_NUMBER() OVER(PARTITION BY Genesys_call_id ORDER BY callDatetime DESC, COALESCE(IS_APC, 0), COALESCE(IS_RPC, 0), COALESCE(IS_PTP, 0))rnk
FROM
  `risk_credit_mis.call_attempt_history_gensys` g,  dt
LEFT JOIN
  `dap_ds_poweruser_playground.campaignmaster` cm
ON
  UPPER(cm.Campaignname) = UPPER(g.campaignName)
INNER JOIN disb ON  SUBSTR(disb.mobileNo, -10) = SUBSTR(CAST(g.mobileNumber AS string), -10)
WHERE
  cm.reference = 'WC'
  and disb.rnk = 1
  and date(callDatetime) between  dt.WkstartDate and dt.WkendDate
)
, 
wc as 
(select wc1.WkstartDate, wc1.WkendDate, Genesys_call_id,
callDatetime,
APC,
RPC,
PTP,
case when APC = 1 then mobileNumber end apc_m,
case when RPC = 1 then mobileNumber end rpc_m,
case when PTP = 1 then mobileNumber end ptp_m,
mobileNumber,
campaignName,
agentGroup,
agentFullName,
employeeId,
connected,
notConnected,
callResult,
rnk,
from wc1 
where rnk = 1
)
,
wcbase as 
( 
select wc.WkstartDate, wc.WkendDate,
 Genesys_call_id,
callDatetime,
APC,
RPC,
PTP,
mobileNumber,
campaignName,
agentGroup,
agentFullName,
employeeId,
connected,
notConnected,
callResult,
rnk,
apc_m,
rpc_m,
ptp_m
from wc
)
, 
cd AS (
  SELECT
    customer_id,
    new_mobile_number,
    ROW_NUMBER() OVER(PARTITION BY customer_id ORDER BY change_date)rnk
  FROM
    `risk_credit_mis.customer_contact_details`
  WHERE
    ACTIVE = 'Y')
  ,
  buck1 as 
  (select distinct dt.WkstartDate, dt.WkendDate, lbfrc.loanAccountNumber, lbfrc.loanStatus, lbfrc.bucketDate, lbfrc.Max_current_DPD as DPD, lmt.customerId, right(cd.new_mobile_number, 10) newmobilenumber
  , row_number() over(partition by lbfrc.loanAccountNumber order by lbfrc.bucketDate desc) buck1rnk
  from `risk_credit_mis.loan_bucket_flow_report_core` lbfrc, dt 
  left join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = lbfrc.loanAccountNumber
  left join cd on cd.customer_id = lmt.customerId
  where date(bucketDate) between  date(dt.WkstartDate) and  date(dt.WkendDate)
  and coalesce(lbfrc.Max_current_DPD, 0) between 1 and 30 
  and date(lbfrc.firstDueDate) <= date(current_date())
  and lbfrc.loanStatus = 'In Arrears'
  and cd.rnk = 1
  )
    ,
buck1cr1 as 
(select distinct
 dt.WkstartDate,
  dt.WkendDate,
  g.Genesys_call_id,
  g.callDatetime,
  COALESCE(g.IS_APC, 0) APC,
  COALESCE(g.IS_RPC, 0) RPC,
  COALESCE(g.IS_PTP,0) PTP,
  RIGHT(g.mobileNumber, 10) mobileNumber,
  buck1.newmobilenumber mobilefrombuckettable,
  g.campaignName,
  g.agentGroup,
  g.agentFullName,
  g.employeeId,
  g.connected,
  g.notConnected,
  g.callResult, 
  buck1.loanAccountNumber,
  buck1.DPD,
  case when g.IS_APC = 1 then g.mobileNumber end apc_m,
  case when g.IS_RPC = 1 then g.mobileNumber end rpc_m,
  case when g.IS_PTP = 1 then mobileNumber end ptp_m,
  ROW_NUMBER() OVER(PARTITION BY g.Genesys_call_id ORDER BY g.callDatetime DESC, COALESCE(g.IS_APC, 0), COALESCE(g.IS_RPC, 0), COALESCE(g.IS_PTP, 0))rnk 
from  `risk_credit_mis.call_attempt_history_gensys` g,  dt
inner join buck1 on buck1.newmobilenumber = right(g.mobileNumber,10)
LEFT JOIN  `dap_ds_poweruser_playground.campaignmaster` cm ON  UPPER(cm.Campaignname) = UPPER(g.campaignName)
where cm.Reference = 'COLL' and cm.Subcategory = 'SOFT'
AND DATE(g.callDatetime) BETWEEN DATE(dt.WkstartDate) AND DATE(dt.WkendDate)
and buck1.buck1rnk = 1
)
,
buck1cr1base as 
(select buck1cr1.WkstartDate, buck1cr1.WkendDate, buck1cr1.Genesys_call_id, buck1cr1.callDatetime
, buck1cr1.mobileNumber
, buck1cr1.mobilefrombuckettable
, buck1cr1.campaignName
, buck1cr1.agentGroup
, buck1cr1.agentFullName
, buck1cr1.employeeId
, buck1cr1.connected
, buck1cr1.notConnected
, buck1cr1.callResult
, buck1cr1.loanAccountNumber
, buck1cr1.apc_m 
, buck1cr1.rpc_m
, buck1cr1.ptp_m 
, buck1cr1.DPD
, buck1cr1.rnk 
 from buck1cr1 where rnk = 1
)
,
buck1end as 
(select distinct dt.WkstartDate, dt.WkendDate, lbfrc.loanAccountNumber, lbfrc.loanStatus, lbfrc.bucketDate, lbfrc.Max_current_DPD as DPD, lmt.customerId, right(cd.new_mobile_number, 10) newmobilenumber
from `risk_credit_mis.loan_bucket_flow_report_core` lbfrc, dt 
left join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = lbfrc.loanAccountNumber
left join cd on cd.customer_id = lmt.customerId
where date(bucketDate) = date(dt.WkendDate)
and date(lbfrc.firstDueDate) <= date(current_date())
and cd.rnk = 1
)
, 
ptpstatus as 
(select max(dt.WkstartDate)WkstartDate, max(dt.WkendDate)WkendDate, cps.Loan_account_no, sum(case when coalesce(cps.PTP_Status, 0) = 1 then 1 else 0 end) ptpstatus 
from `risk_credit_mis.collection_ptp_status`  cps, dt where coalesce(PTP_Status, 0) = 1
and date(contact_date) between dt.WkstartDate and dt.WkendDate
group by cps.Loan_account_no
),
payments as (select loanAccountNumber from `risk_credit_mis.loan_installments_table`, dt where date(lastPaymentDate) between dt.WkstartDate and dt.WkendDate) 
,
buck1normalized as 
(select buck1.WkstartDate, buck1.WkendDate, buck1.loanAccountNumber , buck1.DPD weekstartDPD, buck1end.DPD weekendDPD
, case when buck1.loanAccountNumber in (select distinct Loan_account_no from `risk_credit_mis.collection_ptp_status` where PTP_Due_Date is not null) then buck1.newmobilenumber end ptp_received
, case when buck1.loanAccountNumber in (select distinct Loan_account_no from ptpstatus) then buck1.newmobilenumber end Kept_ptp
, case when coalesce(buck1.DPD, 0) between 1 and 30 and coalesce(buck1end.DPD, 0) = 0 and buck1.loanAccountNumber in (select distinct Loan_account_no from ptpstatus) then buck1.newmobilenumber end Normalized 
, case when coalesce(buck1.DPD, 0) between 1 and 30 and coalesce(buck1end.DPD, 0) = 0 and buck1.loanAccountNumber not in (select distinct Loan_account_no from ptpstatus) then buck1.newmobilenumber end Normalizedwithoutptp 
, case when coalesce(buck1.DPD, 0) between 1 and 30 and coalesce(buck1end.DPD, 0) <= 30 and buck1.loanAccountNumber in (select loanAccountNumber from payments) then buck1.newmobilenumber end Normalized_greg 
, case when coalesce(buck1.DPD, 0) between 1 and 30 then buck1.newmobilenumber end Normalized_greg_denominator
, case when coalesce(buck1.DPD, 0) between 1 and 30 and coalesce(buck1end.DPD, 0) <= 30 and buck1.loanAccountNumber not in (select distinct Loan_account_no from ptpstatus) then buck1.newmobilenumber end Normalizedwithoutptp_greg
from buck1
left join buck1end on buck1.loanAccountNumber =  buck1end.loanAccountNumber
)
,
buck2 as 
(select dt.WkstartDate, dt.WkendDate, lbfrc.loanAccountNumber, lbfrc.loanStatus, lbfrc.bucketDate, lbfrc.Max_current_DPD as DPD, lmt.customerId, right(cd.new_mobile_number, 10) newmobilenumber
, row_number() over(partition by lbfrc.loanAccountNumber order by lbfrc.bucketDate desc) buck2rnk
from `risk_credit_mis.loan_bucket_flow_report_core` lbfrc, dt 
left join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = lbfrc.loanAccountNumber
left join cd on cd.customer_id = lmt.customerId
where date(bucketDate)  between  date(dt.WkstartDate) and  date(dt.WkendDate)
and coalesce(lbfrc.Max_current_DPD, 0) between 31 and 60 
and date(lbfrc.firstDueDate) <= date(current_date())
and lbfrc.loanStatus = 'In Arrears'
and cd.rnk = 1
)
,
buck2cr1 as 
(select   dt.WkstartDate,
  dt.WkendDate,
  g.Genesys_call_id,
  g.callDatetime,
  COALESCE(g.IS_APC, 0) APC,
  COALESCE(g.IS_RPC, 0) RPC,
  COALESCE(g.IS_PTP,0) PTP,
  RIGHT(g.mobileNumber, 10) mobileNumber,
  buck2.newmobilenumber mobilefrombuckettable,
  g.campaignName,
  g.agentGroup,
  g.agentFullName,
  g.employeeId,
  g.connected,
  g.notConnected,
  g.callResult, 
  buck2.loanAccountNumber,
  buck2.DPD,
  case when g.IS_APC = 1 then g.mobileNumber end apc_m,
  case when g.IS_RPC = 1 then g.mobileNumber end rpc_m,
  case when g.IS_PTP = 1 then mobileNumber end ptp_m,
  ROW_NUMBER() OVER(PARTITION BY g.Genesys_call_id ORDER BY g.callDatetime DESC, COALESCE(g.IS_APC, 0), COALESCE(g.IS_RPC, 0), COALESCE(g.IS_PTP, 0))rnk 
from  `risk_credit_mis.call_attempt_history_gensys` g,  dt
inner join buck2 on buck2.newmobilenumber = right(g.mobileNumber,10)
LEFT JOIN
  `dap_ds_poweruser_playground.campaignmaster` cm ON  UPPER(cm.Campaignname) = UPPER(g.campaignName)
where cm.Reference = 'COLL' 
AND DATE(g.callDatetime) BETWEEN DATE(dt.WkstartDate) AND DATE(dt.WkendDate)
and buck2.buck2rnk = 1
)
,buck2cr1base as 
(select buck2cr1.WkstartDate, buck2cr1.WkendDate, buck2cr1.Genesys_call_id, buck2cr1.callDatetime
, buck2cr1.mobileNumber
, buck2cr1.mobilefrombuckettable
, buck2cr1.campaignName
, buck2cr1.agentGroup
, buck2cr1.agentFullName
, buck2cr1.employeeId
, buck2cr1.connected
, buck2cr1.notConnected
, buck2cr1.callResult
, buck2cr1.loanAccountNumber
, buck2cr1.apc_m 
, buck2cr1.rpc_m
, buck2cr1.ptp_m 
, buck2cr1.DPD
, buck2cr1.rnk 
 from buck2cr1 where rnk = 1
)
,
buck2normalized as 
(select buck2.WkstartDate, buck2.WkendDate, buck2.loanAccountNumber , buck2.DPD weekstartDPD, buck1end.DPD weekendDPD
, case when buck2.loanAccountNumber in (select distinct Loan_account_no from `risk_credit_mis.collection_ptp_status` where PTP_Due_Date is not null) then buck2.newmobilenumber end ptp_received
, case when buck2.loanAccountNumber in (select distinct Loan_account_no from ptpstatus) then buck2.newmobilenumber end Kept_ptp
, case when coalesce(buck2.DPD, 0) between 31 and 60 and coalesce(buck1end.DPD, 0) = 0 and buck2.loanAccountNumber in (select distinct Loan_account_no from ptpstatus) then buck2.newmobilenumber end Normalized 
, case when coalesce(buck2.DPD, 0) between 31 and 60 and coalesce(buck1end.DPD, 0) = 0 and buck2.loanAccountNumber not in (select distinct Loan_account_no from ptpstatus) then buck2.newmobilenumber end Normalizedwithoutptp 
, case when coalesce(buck2.DPD, 0) between 31 and 60 then buck2.newmobilenumber end Normalized_greg_denominator 
, case when coalesce(buck2.DPD, 0) between 31 and 60 and coalesce(buck1end.DPD, 0) <= 60 and buck2.loanAccountNumber in (select loanAccountNumber from payments) then buck2.newmobilenumber end Normalized_greg 
, case when coalesce(buck2.DPD, 0) between 31 and 60 and coalesce(buck1end.DPD, 0) <= 60 and buck2.loanAccountNumber not in (select distinct Loan_account_no from ptpstatus) then buck2.newmobilenumber end Normalizedwithoutptp_greg 
from buck2
left join buck1end on buck2.loanAccountNumber =  buck1end.loanAccountNumber
)
,
Ptp_payment as 
(select date_trunc(contact_date, day) contact_date, Loan_account_no, Phone_no, sum(case when coalesce(PTP_Status, 0) > 0 then PTP_Payment_amount else 0 end) ptppaymentamount 
from prj-prod-dataplatform.risk_credit_mis.collection_ptp_status group by date_trunc(contact_date, day), Loan_account_no, Phone_no
)
,
buck1ptpamt as 
(select distinct buck1normalized.WkstartDate,buck1normalized.WkendDate,loanAccountNumber
, case when Normalized_greg is not null then (select sum(ptppaymentamount) from Ptp_payment where Loan_account_no = buck1normalized.loanAccountNumber and date(contact_date) between dt.WkstartDate and dt.WkendDate) else 0 end ptpamount_normalized  
,(select sum(ptppaymentamount) from Ptp_payment where Loan_account_no = buck1normalized.loanAccountNumber and date(contact_date) between dt.WkstartDate and dt.WkendDate)  ptpamount  
from buck1normalized, dt)
,
buck2ptpamt as 
(select distinct buck2normalized.WkstartDate,buck2normalized.WkendDate,loanAccountNumber
, case when Normalized is not null then (select sum(ptppaymentamount) from Ptp_payment where Loan_account_no = buck2normalized.loanAccountNumber and date(contact_date) 
between dt.WkstartDate and dt.WkendDate) else 0 end ptpamount_normalized  
, (select sum(ptppaymentamount) from Ptp_payment where Loan_account_no = buck2normalized.loanAccountNumber and date(contact_date) 
between dt.WkstartDate and dt.WkendDate)  ptpamount 
from buck2normalized, dt),
fte as 
(select distinct agentFullName FROM
  `risk_credit_mis.call_attempt_history_gensys` g,
  dt
LEFT JOIN
  `dap_ds_poweruser_playground.campaignmaster` cm
ON
  UPPER(cm.Campaignname) = UPPER(g.campaignName)
WHERE
   DATE(g.callDatetime) BETWEEN DATE(dt.WkstartDate)  AND DATE(WkendDate)
   and cm.reference in ('REM', 'COLL', 'WC')),
LUM as 
(select distinct loanAccountNumber from `prj-prod-dataplatform.risk_credit_mis.loan_master_table`, dt where flagDisbursement = 1 and date_trunc(disbursementDateTime, day) < dt.WkstartDate
and loanPaidStatus not in ('Written Off','Completed', 'Settled')),
aw as 
(select bucketDate, loanAccountNumber from `risk_credit_mis.loan_bucket_flow_report_core`, dt where coalesce(Max_current_DPD ,0) between 1 and 179 and date(bucketDate) = dt.WkstartDate and loanStatus not in ('Written Off','Completed', 'Settled'))
select *,
(select count(distinct ptp_received) from buck1normalized)+(select count(distinct ptp_received) from buck2normalized) ptp_received,
(select count(distinct Kept_ptp) from buck1normalized) + (select count(distinct Kept_ptp) from buck2normalized) kept_ptp,
(select sum(ptpamount) from buck1ptpamt)bucket1ptppmt,
(select sum(ptpamount) from buck2ptpamt) bucket2ptppmt,
(select count(distinct agentFullName) from fte)FTE,
(select count(distinct loanAccountNumber) from LUM) total_accounts,
(select count(distinct loanAccountNumber) from aw) active_works
from dt;
"""

df_others = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
df_others['kept_ptp_rate'] = df_others['kept_ptp']/df_others['ptp_received']
df_others.columns

Job ID 269d07e7-ebd2-4096-8b2a-4b299e95c4c4 successfully executed: |          |
Downloading: 100%|██████████|


Index(['WkstartDate', 'WkendDate', 'ptp_received', 'kept_ptp', 'bucket1ptppmt',
       'bucket2ptppmt', 'FTE', 'total_accounts', 'active_works',
       'kept_ptp_rate'],
      dtype='object')

In [41]:
pd.set_option("display.max_columns", None)
df = pd.concat([df, df_others[['ptp_received', 'kept_ptp', 'kept_ptp_rate', 'bucket1ptppmt',
       'bucket2ptppmt', 'FTE', 'total_accounts', 'active_works'    ]]], axis = 1)
df

,WkstartDate,WkendDate,apc_wc_connected,wccalled,WelcomeCall_ContactRate,apc_rc_connected,rccalled,remindercallcontactrate,bucket1contacted,bucketcallcontacted,bucket1contactrate,Normalized_Greg,Normalized_denominator,Bucket1_Normalized,bucket2contacted,bucket2callcontacted,bucket2contactrate,Normalized_buck2_Greg,Normalized_buck2_denominator,bucket2Normalizationrate,Bucket1_inArearstoTotal_Overdue_Amount,Bucket1_paymentsfromoverdue,Bucket1_recoveryrate,Bucket2_inArearstoTotal_Overdue_Amount,Bucket2_paymentsfromoverdue,Bucket2_recoveryrate,Bucket3_inArearstoTotal_Overdue_Amount,Bucket3_paymentsfromoverdue,Bucket3_recoveryrate,Bucket4_inArearstoTotal_Overdue_Amount,Bucket4_paymentsfromoverdue,Bucket4_recoveryrate,Bucket5_inArearstoTotal_Overdue_Amount,Bucket5_paymentsfromoverdue,Bucket5_recoveryrate,Bucket6_inArearstoTotal_Overdue_Amount,Bucket6_paymentsfromoverdue,Bucket6_recoveryrate,Bucket7_inArearstoTotal_Overdue_Amount,Bucket7_paymentsfromoverdue,Bucket7_recoveryrate,Bucket8_inArearstoTotal_Overdue_Amount,Bucket8_paymentsfromoverdue,Bucket8_recoveryrate,inArearstoTotal_Overdue_Amount,paymentsfromoverdue,recoveryrate,wentdelinquent,rollbacked,defaulteddelinquentamount,rollbacked_amount,rollback_rate,rollback_rate_v,ptp_received,kept_ptp,kept_ptp_rate,bucket1ptppmt,bucket2ptppmt,FTE,total_accounts,active_works
0,2024-04-22,2024-04-28,1925,3061,0.628879,3314,8464,0.391541,723,4343,0.166475,530,3472,0.15265,177,2466,0.071776,101,2172,0.046501,8536882.89,1197540.95,0.140278,10904360.07,438544.86,0.040217,13648845.42,159959.06,0.01172,14423530.02,131809.91,0.009139,17763422.98,89090.35,0.005015,25048815.96,76136.13,0.00304,1.529429e+08,17668.87,0.000116,196705449.0,NaN,NaN,4.399742e+08,3353651.01,0.007622,2884,430,6278667.8,892494.13,0.149098,0.142147,2370,271,0.114346,519219.21,85414.38,31,63496,11676


This SQL code performs a complex analysis involving multiple steps to derive various metrics related to call center operations, loan accounts, and payment status. Let's break down the logic of the code:

1. **Common Table Expressions (CTEs):**
   - **`dt`:** Creates a date table with the provided start and end dates.
   - **`disb`:** Retrieves disbursement-related information for loans, filtering out settled or completed loans.
   - **`wc1`:** Retrieves call attempt history data from the Genesys system, joining it with the `disb` CTE based on mobile numbers and campaign references. It uses window functions to assign row numbers based on call datetime and various call types (APC, RPC, PTP).
   - **`wc`:** Filters the `wc1` CTE to include only rows with row number (`rnk`) equal to 1, effectively selecting the latest call attempt for each Genesys call ID.
   - **`wcbase`:** Creates a base table for call attempt history with relevant columns.

2. **Customer Contact Details CTE (`cd`):**
   - Retrieves customer contact details and assigns row numbers based on the change date.

3. **Bucket 1 CTE (`buck1`):**
   - Retrieves loan bucket flow data for loans with a maximum DPD between 1 and 30, first due date before the current date, and an 'In Arrears' status.

4. **Bucket 1 Call Attempt (`buck1cr1`):**
   - Retrieves call attempt history for Bucket 1 loans, filtering based on campaign references and call datetime. It also uses window functions to assign row numbers.

5. **Bucket 1 Normalization (`buck1normalized`):**
   - Performs normalization for Bucket 1, categorizing loans based on specific conditions and calculating certain metrics.

6. **Bucket 2 CTE (`buck2`):**
   - Retrieves loan bucket flow data for loans with a maximum DPD between 31 and 60, 'In Arrears' status, and first due date before the current date.

7. **Bucket 2 Call Attempt (`buck2cr1`):**
   - Retrieves call attempt history for Bucket 2 loans, filtering based on campaign references and call datetime. It also uses window functions to assign row numbers.

8. **Bucket 2 Normalization (`buck2normalized`):**
   - Performs normalization for Bucket 2, categorizing loans based on specific conditions and calculating certain metrics.

9. **PTP Status CTE (`ptpstatus`):**
   - Determines the PTP (Promise to Pay) status for loans within the specified date range.

10. **Payments CTE (`payments`):**
    - Retrieves loan account numbers with payments made within the specified date range.

11. **Bucket 1 PTP Amount (`buck1ptpamt`):**
    - Calculates PTP amounts for Bucket 1 loans.

12. **Bucket 2 PTP Amount (`buck2ptpamt`):**
    - Calculates PTP amounts for Bucket 2 loans.

13. **Full-Time Employees (`fte`):**
    - Retrieves distinct agent full names considered as full-time employees.

14. **Loan Under Management (`LUM`):**
    - Retrieves loan account numbers with a disbursement date before the current analysis period.

15. **Active Works (`aw`):**
    - Retrieves active works (loans) with a maximum DPD between 1 and 179.

16. **Final Selection and Aggregation:**
    - The final SELECT statement aggregates various metrics and counts based on the derived CTEs and external tables.

17. **Derived Metrics:**
    - Calculations for metrics like PTP received, kept PTP, PTP payment amounts, full-time employees, total accounts, and active works.

18. **Calculated Rates:**
    - Calculates the 'kept PTP rate' as the ratio of kept PTP to PTP received.

The code essentially combines data from multiple sources, performs various normalizations and aggregations, and calculates key metrics related to call center operations, loan accounts, and payment status. The final output is a DataFrame with the derived metrics and rates.

# Final Merge

In [42]:
df['WkstartDate'] = df['WkstartDate'].astype(str)
df['WkendDate'] = df['WkendDate'].astype(str)
# Convert columns to the correct data types
df['bucket1ptppmt'] = df['bucket1ptppmt'].astype(float)
df['bucket2ptppmt'] = df['bucket2ptppmt'].astype(float)
df['WelcomeCall_ContactRate'] = df['WelcomeCall_ContactRate'].astype(float)
df['remindercallcontactrate'] = df['remindercallcontactrate'].astype(float)
df['bucket1contactrate'] = df['bucket1contactrate'].astype(float)
df['Bucket1_Normalized'] = df['Bucket1_Normalized'].astype(float)
df['bucket2contactrate'] = df['bucket2contactrate'].astype(float)
df['bucket2Normalizationrate'] = df['bucket2Normalizationrate'].astype(float)
df['kept_ptp_rate'] = df['kept_ptp_rate'].astype(float)
df['Bucket1_recoveryrate'] = df['Bucket1_recoveryrate'].astype(float)
df['Bucket2_recoveryrate'] = df['Bucket2_recoveryrate'].astype(float)
df['Bucket3_recoveryrate'] = df['Bucket3_recoveryrate'].astype(float)
df['Bucket4_recoveryrate'] = df['Bucket4_recoveryrate'].astype(float)
df['Bucket5_recoveryrate'] = df['Bucket5_recoveryrate'].astype(float)
df['Bucket6_recoveryrate'] = df['Bucket6_recoveryrate'].astype(float)
df['Bucket7_recoveryrate'] = df['Bucket7_recoveryrate'].astype(float)
df['Bucket8_recoveryrate'] = df['Bucket8_recoveryrate'].astype(float)
df['recoveryrate'] = df['recoveryrate'].astype(float)
df['paymentsfromoverdue'] = df['paymentsfromoverdue'].astype(float)
df['inArearstoTotal_Overdue_Amount'] = df['inArearstoTotal_Overdue_Amount'].astype(float)
df['defaulteddelinquentamount'] = df['defaulteddelinquentamount'].astype(float)	
df['rollbacked_amount'] = df['rollbacked_amount'].astype(float)
df['rollback_rate'] = df['rollback_rate'].astype(float)
df['rollback_rate_v'] = df['rollback_rate_v'].astype(float)
			



df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 61 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   WkstartDate                             1 non-null      object 
 1   WkendDate                               1 non-null      object 
 2   apc_wc_connected                        1 non-null      Int64  
 3   wccalled                                1 non-null      Int64  
 4   WelcomeCall_ContactRate                 1 non-null      float64
 5   apc_rc_connected                        1 non-null      Int64  
 6   rccalled                                1 non-null      Int64  
 7   remindercallcontactrate                 1 non-null      float64
 8   bucket1contacted                        1 non-null      Int64  
 9   bucketcallcontacted                     1 non-null      Int64  
 10  bucket1contactrate                      1 non-null      float64
 1

In [43]:
df.head()

,WkstartDate,WkendDate,apc_wc_connected,wccalled,WelcomeCall_ContactRate,apc_rc_connected,rccalled,remindercallcontactrate,bucket1contacted,bucketcallcontacted,bucket1contactrate,Normalized_Greg,Normalized_denominator,Bucket1_Normalized,bucket2contacted,bucket2callcontacted,bucket2contactrate,Normalized_buck2_Greg,Normalized_buck2_denominator,bucket2Normalizationrate,Bucket1_inArearstoTotal_Overdue_Amount,Bucket1_paymentsfromoverdue,Bucket1_recoveryrate,Bucket2_inArearstoTotal_Overdue_Amount,Bucket2_paymentsfromoverdue,Bucket2_recoveryrate,Bucket3_inArearstoTotal_Overdue_Amount,Bucket3_paymentsfromoverdue,Bucket3_recoveryrate,Bucket4_inArearstoTotal_Overdue_Amount,Bucket4_paymentsfromoverdue,Bucket4_recoveryrate,Bucket5_inArearstoTotal_Overdue_Amount,Bucket5_paymentsfromoverdue,Bucket5_recoveryrate,Bucket6_inArearstoTotal_Overdue_Amount,Bucket6_paymentsfromoverdue,Bucket6_recoveryrate,Bucket7_inArearstoTotal_Overdue_Amount,Bucket7_paymentsfromoverdue,Bucket7_recoveryrate,Bucket8_inArearstoTotal_Overdue_Amount,Bucket8_paymentsfromoverdue,Bucket8_recoveryrate,inArearstoTotal_Overdue_Amount,paymentsfromoverdue,recoveryrate,wentdelinquent,rollbacked,defaulteddelinquentamount,rollbacked_amount,rollback_rate,rollback_rate_v,ptp_received,kept_ptp,kept_ptp_rate,bucket1ptppmt,bucket2ptppmt,FTE,total_accounts,active_works
0,2024-04-22,2024-04-28,1925,3061,0.628879,3314,8464,0.391541,723,4343,0.166475,530,3472,0.15265,177,2466,0.071776,101,2172,0.046501,8536882.89,1197540.95,0.140278,10904360.07,438544.86,0.040217,13648845.42,159959.06,0.01172,14423530.02,131809.91,0.009139,17763422.98,89090.35,0.005015,25048815.96,76136.13,0.00304,1.529429e+08,17668.87,0.000116,196705449.0,NaN,NaN,4.399742e+08,3353651.01,0.007622,2884,430,6278667.8,892494.13,0.149098,0.142147,2370,271,0.114346,519219.21,85414.38,31,63496,11676


In [44]:
# Reset the index of the DataFrame
df = df.reset_index(drop=True)

# Define the dataset and table references
dataset_id = 'dap_ds_poweruser_playground'
table_id = 'collection_Weekly_kpi_v6'
# Define the table schema as per your DataFrame columns
schema = [
    bigquery.SchemaField("WkstartDate", "string"),
    bigquery.SchemaField("WkendDate", "string"),
    bigquery.SchemaField("apc_wc_connected", "int64"),
    bigquery.SchemaField("wccalled", "int64"),
    bigquery.SchemaField("WelcomeCall_ContactRate", "float64"),
    bigquery.SchemaField("apc_rc_connected", "int64"),
    bigquery.SchemaField("rccalled", "int64"),
    bigquery.SchemaField("remindercallcontactrate", "float64"),
    bigquery.SchemaField("bucket1contacted", "int64"),
    bigquery.SchemaField("bucketcallcontacted", "int64"),
    bigquery.SchemaField("bucket1contactrate", "float64"),
    bigquery.SchemaField("Normalized_Greg", "int64"),
    bigquery.SchemaField("Normalized_denominator", "int64"),
    bigquery.SchemaField("Bucket1_Normalized", "float64"),
    bigquery.SchemaField("bucket2contacted", "int64"),
    bigquery.SchemaField("bucket2callcontacted", "int64"),
    bigquery.SchemaField("bucket2contactrate", "float64"),
    bigquery.SchemaField("Normalized_buck2_Greg", "int64"),
    bigquery.SchemaField("Normalized_buck2_denominator", "int64"),
    bigquery.SchemaField("bucket2Normalizationrate", "float64"),
    bigquery.SchemaField("ptp_received", "int64"),
    bigquery.SchemaField("kept_ptp", "int64"),
    bigquery.SchemaField("kept_ptp_rate", "float64"),
    bigquery.SchemaField("inArearstoTotal_Overdue_Amount", "float64"),
    bigquery.SchemaField("paymentsfromoverdue", "float64"),
    bigquery.SchemaField("Bucket1_recoveryrate", 'float64'),
    bigquery.SchemaField("Bucket2_recoveryrate", 'float64'),
    bigquery.SchemaField("Bucket3_recoveryrate", 'float64'),
    bigquery.SchemaField("Bucket4_recoveryrate", 'float64'),
    bigquery.SchemaField("Bucket5_recoveryrate", 'float64'),
    bigquery.SchemaField("Bucket6_recoveryrate", 'float64'),
    bigquery.SchemaField("Bucket7_recoveryrate", 'float64'),
    bigquery.SchemaField("Bucket8_recoveryrate", 'float64'),
    bigquery.SchemaField("recoveryrate", 'float64'),
    bigquery.SchemaField("wentdelinquent", "int64"),
    bigquery.SchemaField("rollbacked", "int64"),
    bigquery.SchemaField("defaulteddelinquentamount", "float64"),
    bigquery.SchemaField("rollbacked_amount", "float64"),
    bigquery.SchemaField("rollback_rate", "float64"),
    bigquery.SchemaField("rollback_rate_v", "float64"),
    bigquery.SchemaField("bucket1ptppmt", "float64"),
    bigquery.SchemaField("bucket2ptppmt", "float64"),
    bigquery.SchemaField("FTE", "int64"),
    bigquery.SchemaField("total_accounts", "int64"),
    bigquery.SchemaField("active_works", "int64"),
]
# Create the dataset reference
dataset_ref = client.dataset(dataset_id)

# Define the table reference
table_ref = dataset_ref.table(table_id)

# Configure the job to overwrite the table if it already exists
job_config = bigquery.LoadJobConfig(schema = schema)

# Load the DataFrame into BigQuery
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)

# Wait for the job to complete
job.result()

print(f"Table {table_id} created in dataset {dataset_id}.")

Table collection_Weekly_kpi_v6 created in dataset dap_ds_poweruser_playground.


In [45]:
sq = """select * from dap_ds_poweruser_playground.collection_Weekly_kpi_v6 order by WkstartDate desc;"""
dfd = client.query(sq).to_dataframe()
dfd

,WkstartDate,WkendDate,apc_wc_connected,wccalled,WelcomeCall_ContactRate,apc_rc_connected,rccalled,remindercallcontactrate,bucket1contacted,bucketcallcontacted,bucket1contactrate,Normalized_Greg,Normalized_denominator,Bucket1_Normalized,bucket2contacted,bucket2callcontacted,bucket2contactrate,Normalized_buck2_Greg,Normalized_buck2_denominator,bucket2Normalizationrate,Bucket1_inArearstoTotal_Overdue_Amount,Bucket1_paymentsfromoverdue,Bucket1_recoveryrate,Bucket2_inArearstoTotal_Overdue_Amount,Bucket2_paymentsfromoverdue,Bucket2_recoveryrate,Bucket3_inArearstoTotal_Overdue_Amount,Bucket3_paymentsfromoverdue,Bucket3_recoveryrate,Bucket4_inArearstoTotal_Overdue_Amount,Bucket4_paymentsfromoverdue,Bucket4_recoveryrate,Bucket5_inArearstoTotal_Overdue_Amount,Bucket5_paymentsfromoverdue,Bucket5_recoveryrate,Bucket6_inArearstoTotal_Overdue_Amount,Bucket6_paymentsfromoverdue,Bucket6_recoveryrate,Bucket7_inArearstoTotal_Overdue_Amount,Bucket7_paymentsfromoverdue,Bucket7_recoveryrate,Bucket8_inArearstoTotal_Overdue_Amount,Bucket8_paymentsfromoverdue,Bucket8_recoveryrate,inArearstoTotal_Overdue_Amount,paymentsfromoverdue,recoveryrate,wentdelinquent,rollbacked,defaulteddelinquentamount,rollbacked_amount,rollback_rate,rollback_rate_v,ptp_received,kept_ptp,kept_ptp_rate,bucket1ptppmt,bucket2ptppmt,FTE,total_accounts,active_works
0,2024-04-22,2024-04-28,1925,3061,0.628879,3314,8464,0.391541,723,4343,0.166475,530,3472,0.152650,177,2466,0.071776,101,2172,0.046501,8536882.89,1197540.95,0.140278,10904360.07,438544.86,0.040217,13648845.42,159959.06,0.011720,14423530.02,131809.91,0.009139,17763422.98,89090.35,0.005015,25048815.96,76136.13,0.003040,1.529429e+08,17668.87,0.000116,1.967054e+08,NaN,NaN,4.399742e+08,3353651.01,0.007622,2884,430,6278667.80,892494.13,0.149098,0.142147,2370,271,0.114346,519219.21,85414.38,31,63496,11676
1,2024-04-15,2024-04-21,1623,2548,0.636970,3064,7923,0.386722,913,5172,0.176527,1195,4182,0.285748,162,2514,0.064439,92,2112,0.043561,10105945.11,3085073.20,0.305273,10521717.23,352729.64,0.033524,13349467.23,148167.92,0.011099,14681277.93,147726.21,0.010062,18448603.03,121152.24,0.006567,25629179.13,117398.02,0.004581,1.637868e+08,83017.07,0.000507,2.729865e+08,NaN,NaN,5.295094e+08,5227058.49,0.009872,7667,1235,18109072.53,3059172.17,0.161080,0.168930,2689,407,0.151357,892246.37,31594.02,29,58057,12208
2,2024-04-08,2024-04-14,1407,2380,0.591176,3475,8204,0.423574,632,3938,0.160488,395,3142,0.125716,140,2647,0.052890,94,2205,0.042630,7915484.23,998595.62,0.126157,10980901.66,408874.04,0.037235,12348360.22,155807.57,0.012618,14615239.59,103603.50,0.007089,19300464.22,95420.94,0.004944,27573482.98,59610.18,0.002162,1.554976e+08,171520.07,0.001103,2.680638e+08,NaN,NaN,5.162953e+08,2951443.76,0.005717,2238,318,4995925.30,660478.49,0.142091,0.132203,2269,185,0.081534,333890.13,57871.03,29,58211,11281
3,2024-04-01,2024-04-07,1095,1981,0.552751,2285,6917,0.330346,975,4758,0.204918,952,3661,0.260038,148,2309,0.064097,106,2119,0.050024,8665518.31,2418621.27,0.279109,10129976.00,423298.51,0.041787,12590391.19,215236.14,0.017095,14271989.22,97342.39,0.006821,19989514.52,82661.35,0.004135,25836659.14,31540.45,0.001221,1.559157e+08,76909.34,0.000493,2.631120e+08,NaN,NaN,5.105118e+08,4754147.35,0.009313,7178,1312,16937371.59,3007508.21,0.182781,0.177566,2573,460,0.178780,866519.50,62598.02,27,56532,11708
4,2024-03-25,2024-03-31,334,601,0.555740,2789,7191,0.387846,449,3623,0.123930,729,3600,0.202500,131,2627,0.049867,76,1974,0.038501,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000e+00,0.00,0.000000,0.000000e+00,0.0,0.0,4.963229e+08,3150064.47,0.006347,5240,388,12347556.06,920889.32,0.074046,0.074581,2164,149,0.068854,248999.11,13224.16,28,55742,11595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,